In [2]:
%matplotlib inline

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib, collections, datetime, hashlib, os, re, time, sys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 对DOH进行分析,扫描时间2021-12-21

# 合并所有文件

In [3]:
tls_raw = pd.read_json("./tls-cert-ct-revoke-scsv/result_tls.txt",lines=True)
cert_raw = pd.read_json("./tls-cert-ct-revoke-scsv/result_cert.txt",lines=True)
ct_raw = pd.read_json("./tls-cert-ct-revoke-scsv/result_ct.txt",lines=True)
revoke_raw = pd.read_json("./tls-cert-ct-revoke-scsv/result_revoke.txt",lines=True)
tlsa_caa_raw = pd.read_json("./tlsa-caa/result_tlsa_caa.txt",lines=True)
caa_analysis_raw = pd.read_csv("./tlsa-caa/caa_analysis.txt")
dane_raw = pd.read_csv("./dane/result_dane_valid.txt")
tls13_down = pd.read_json("./tls13down/result_down.txt",lines=True)


all_raw = pd.merge(tls_raw,cert_raw)
all_raw = pd.merge(all_raw,ct_raw)
all_raw = pd.merge(all_raw,revoke_raw)
all_raw = pd.merge(all_raw,caa_analysis_raw,how = "outer")
all_raw = pd.merge(all_raw,tlsa_caa_raw,how = "outer")
all_raw = pd.merge(all_raw,dane_raw,how = "outer")
all_raw = pd.merge(all_raw,tls13_down,how = "outer")
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tlsa_dnssec,caa_dnssec,tlsa_error,caa_error,dnssec,dane,err,tls13_down_12,tls13_down_11,tls13_down_10
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,,,,,NaN,NaN,NaN,True,True,True
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,False,True,True
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,True,False,False
3,freedns.controld.dev,176.125.239.51,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,True,False,False
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,,,,,NaN,NaN,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134828,freedns.controld.com,76.76.10.144,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,True,False,False
134829,freedns.controld.com,76.76.10.247,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,True,False,False
134830,premium.dns.controld.com,76.76.2.127,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,True,False,False
134831,dns.controld.com,76.76.2.206,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,,,NaN,NaN,NaN,True,False,False


In [4]:
import gc
del tls_raw,cert_raw,ct_raw,revoke_raw,tlsa_caa_raw,caa_analysis_raw,dane_raw,tls13_down
gc.collect()

0

## 合并HTTP头 

In [5]:
htttpheader_raw = pd.read_csv("./httpheader/http.csv")
htttpheader_raw = pd.DataFrame(htttpheader_raw,columns = ['host','server_name','http_code','http_headers'])
htttpheader_raw = htttpheader_raw.rename(columns={"host":"ip","server_name":"domain"})
htttpheader_raw

,ip,domain,http_code,http_headers
0,1.0.0.1,NaN,400,NaN
1,1.0.0.1,NaN,400,NaN
2,1.0.0.3,NaN,400,NaN
3,1.0.0.2,NaN,400,NaN
4,101.198.192.33,doh.360.cn,302,NaN
...,...,...,...,...
7138,176.125.239.86,dns.controld.dev,404,NaN
7139,87.228.229.146,dns.google.com,302,Strict-Transport-Security: max-age=31536000; i...
7140,45.91.92.121,NaN,-1,NaN
7141,78.46.244.143,NaN,-1,NaN


In [6]:
all_raw = pd.merge(all_raw,htttpheader_raw,how="left",on=['ip','domain'])
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tlsa_error,caa_error,dnssec,dane,err,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,,,NaN,NaN,NaN,True,True,True,200.0,Strict-Transport-Security: max-age=31536000; i...
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,False,True,True,404.0,NaN
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
3,freedns.controld.dev,176.125.239.51,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,,,NaN,NaN,NaN,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134828,freedns.controld.com,76.76.10.144,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,NaN,NaN
134829,freedns.controld.com,76.76.10.247,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,NaN,NaN
134830,premium.dns.controld.com,76.76.2.127,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,NaN,NaN
134831,dns.controld.com,76.76.2.206,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,NaN,NaN


## 合并公司和ISP 

In [7]:
location_raw = pd.read_json("../../map/doh/doh-location-ip.txt",lines=True)
# location_raw = pd.DataFrame(location_raw,columns = ['query','country','isp','org'])
location_raw = location_raw.rename(columns={"query":"ip"})
location_raw

,status,country,countryCode,region,regionName,city,zip,lat,lon,timezone,isp,org,as,ip
0,success,Australia,AU,QLD,Queensland,South Brisbane,4101,-27.47660,153.01660,Australia/Brisbane,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,"AS13335 Cloudflare, Inc.",1.0.0.1
1,success,Australia,AU,QLD,Queensland,South Brisbane,4101,-27.47660,153.01660,Australia/Brisbane,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,"AS13335 Cloudflare, Inc.",1.0.0.2
2,success,Australia,AU,QLD,Queensland,South Brisbane,4101,-27.47660,153.01660,Australia/Brisbane,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,"AS13335 Cloudflare, Inc.",1.0.0.3
3,success,Singapore,SG,,,Singapore,,1.31857,103.95000,Asia/Singapore,Myrepublic Limited,Republictelecom,AS56300 MyRepublic Ltd.,101.100.188.23
4,success,Taiwan,TW,,Taiwan,Taipei,,25.03300,121.56500,Asia/Taipei,Taiwan Network Information Center,Twnic Experimental,AS131621 Taiwan Network Information Center,101.101.101.101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9329,success,Japan,JP,13,Tokyo,Shibuya,150-8307,35.65650,139.70000,Asia/Tokyo,GMO-Z.COM PTE. LTD.,"GMO Internet, Inc.",AS59349 GMO-Z.COM PTE. LTD.,163.44.153.212
9330,success,Nigeria,NG,LA,Lagos,Ikeja,,6.58849,3.36369,Africa/Lagos,GARANNTOR,,AS328110 Garanntor Hosting Limited,164.160.130.158
9331,success,United States,US,CA,California,Santa Clara,95054,37.39310,-121.96200,America/Los_Angeles,"DigitalOcean, LLC","DigitalOcean, LLC","AS14061 DigitalOcean, LLC",164.90.155.71
9332,success,Germany,DE,HE,Hesse,Frankfurt am Main,60313,50.11880,8.68430,Europe/Berlin,"DigitalOcean, LLC","DigitalOcean, LLC","AS14061 DigitalOcean, LLC",164.90.161.84


In [8]:
test = pd.read_csv("./raw_data/result_doh.txt")
test

,ip,suffix,resflag,geth1,geth2,posth1,posth2,jsonh1,jsonh2
0,1.0.0.2,dns-query,True,True,True,True,True,False,False
1,130.61.56.82,dns-query,True,True,True,True,True,False,False
2,135.84.249.13,dns-query,True,False,False,True,True,False,False
3,135.84.249.13,query,True,False,False,False,True,False,False
4,135.84.249.13,resolve,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...
23190,52.42.134.63,dns-query,True,True,True,True,True,False,False
23191,63.240.152.245,dns-query,True,True,True,True,True,False,False
23192,63.240.152.245,query,True,True,True,True,True,False,False
23193,63.240.152.245,resolve,True,True,True,True,True,False,False


In [9]:
test = pd.merge(test,location_raw,how = "left",on=['ip'])
test

,ip,suffix,resflag,geth1,geth2,posth1,posth2,jsonh1,jsonh2,status,...,region,regionName,city,zip,lat,lon,timezone,isp,org,as
0,1.0.0.2,dns-query,True,True,True,True,True,False,False,success,...,QLD,Queensland,South Brisbane,4101,-27.4766,153.0166,Australia/Brisbane,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,"AS13335 Cloudflare, Inc."
1,130.61.56.82,dns-query,True,True,True,True,True,False,False,success,...,HE,Hesse,Frankfurt am Main,60326,50.1049,8.6295,Europe/Berlin,Oracle Corporation,Oracle Cloud Infrastructure (eu-frankfurt-1),AS31898 Oracle Corporation
2,135.84.249.13,dns-query,True,False,False,True,True,False,False,success,...,KA,Karnataka,Mangalore,575001,12.8696,74.8453,Asia/Kolkata,"iZus Co., Ltd",Scape Reach LTD,AS58810 IZUSCOLTD-BN
3,135.84.249.13,query,True,False,False,False,True,False,False,success,...,KA,Karnataka,Mangalore,575001,12.8696,74.8453,Asia/Kolkata,"iZus Co., Ltd",Scape Reach LTD,AS58810 IZUSCOLTD-BN
4,135.84.249.13,resolve,True,False,False,True,False,False,False,success,...,KA,Karnataka,Mangalore,575001,12.8696,74.8453,Asia/Kolkata,"iZus Co., Ltd",Scape Reach LTD,AS58810 IZUSCOLTD-BN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23190,52.42.134.63,dns-query,True,True,True,True,True,False,False,success,...,OR,Oregon,Portland,97207,45.5235,-122.6760,America/Los_Angeles,"Amazon.com, Inc.",AWS EC2 (us-west-2),"AS16509 Amazon.com, Inc."
23191,63.240.152.245,dns-query,True,True,True,True,True,False,False,success,...,WI,Wisconsin,Kenosha,53141,42.5842,-87.8216,America/Chicago,CERFnet,Snap-on Incorporated,AS17229 AT&T Enhanced Network Services
23192,63.240.152.245,query,True,True,True,True,True,False,False,success,...,WI,Wisconsin,Kenosha,53141,42.5842,-87.8216,America/Chicago,CERFnet,Snap-on Incorporated,AS17229 AT&T Enhanced Network Services
23193,63.240.152.245,resolve,True,True,True,True,True,False,False,success,...,WI,Wisconsin,Kenosha,53141,42.5842,-87.8216,America/Chicago,CERFnet,Snap-on Incorporated,AS17229 AT&T Enhanced Network Services


In [10]:
test["org"] = test.org.apply(lambda x: x.lower())
test

,ip,suffix,resflag,geth1,geth2,posth1,posth2,jsonh1,jsonh2,country,isp,org
0,1.0.0.2,dns-query,True,True,True,True,True,False,False,Australia,"Cloudflare, Inc",apnic and cloudflare dns resolver project
1,130.61.56.82,dns-query,True,True,True,True,True,False,False,Germany,Oracle Corporation,oracle cloud infrastructure (eu-frankfurt-1)
2,135.84.249.13,dns-query,True,False,False,True,True,False,False,India,"iZus Co., Ltd",scape reach ltd
3,135.84.249.13,query,True,False,False,False,True,False,False,India,"iZus Co., Ltd",scape reach ltd
4,135.84.249.13,resolve,True,False,False,True,False,False,False,India,"iZus Co., Ltd",scape reach ltd
...,...,...,...,...,...,...,...,...,...,...,...,...
23190,52.42.134.63,dns-query,True,True,True,True,True,False,False,United States,"Amazon.com, Inc.",aws ec2 (us-west-2)
23191,63.240.152.245,dns-query,True,True,True,True,True,False,False,United States,CERFnet,snap-on incorporated
23192,63.240.152.245,query,True,True,True,True,True,False,False,United States,CERFnet,snap-on incorporated
23193,63.240.152.245,resolve,True,True,True,True,True,False,False,United States,CERFnet,snap-on incorporated


In [11]:
for index,value in test["org"].value_counts().iteritems():
    if value >= 20:
        print(index,value,value/len(test)*100) 

nextdns, inc 4578 19.73701228713085
nextdns, inc. 3780 16.296615649924554
scape reach ltd 3071 13.239922397068336
controld inc 1536 6.622116835524898
cony brake co., ltd 1339 5.772795861176978
 785 3.3843500754472946
zenu software llc 738 3.1817201983186028
win soon lee co., ltd 706 3.0437594309118343
eagle sky co., lt 441 1.9012718258245311
snap-on tools corpattn purch 388 1.6727743048070702
snap-on incorporated 248 1.0691959474024575
aliyun computing co., ltd 189 0.8148307824962276
anx 172 0.7415391248113818
woodynet 156 0.6725587411079974
vultr holdings, llc 127 0.5475317956456133
kornet 119 0.5130416037939212
cegetel 116 0.5001077818495365
iij internet 110 0.47424013796076736
digitalocean, llc 110 0.47424013796076736
aws ec2 (us-east-1) 109 0.4699288639793059
telia network services 87 0.3750808363871524
aws ec2 (us-west-2) 75 0.3233455486096141
linode 74 0.31903427462815265
hetzner 57 0.24574261694330674
avions transport regional 52 0.22418624703599913
gedas operational services gm

In [12]:
print("doh公司 数量 占比")
nextdns = test[test["org"].str.contains("nextdns")]
print("nextdns",len(nextdns), len(nextdns)/len(test)*100)
scape = test[test["org"].str.contains("scape")]
print("scape reach",len(scape), len(scape)/len(test)*100)
controld = test[test["org"].str.contains("controld")]
print("controld",len(controld), len(controld)/len(test)*100)
cony = test[test["org"].str.contains("cony")]
print("cony brake",len(cony), len(cony)/len(test)*100)
zenu = test[test["org"].str.contains("zenu")]
print("zenu software",len(zenu), len(zenu)/len(test)*100)

doh公司 数量 占比
nextdns 8511 36.69325285621901
scape reach 3075 13.25716749299418
controld 1536 6.622116835524898
cony brake 1339 5.772795861176978
zenu software 738 3.1817201983186028


In [32]:
all_raw = pd.merge(all_raw,location_raw,how = "left",on=['ip'])
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,region,regionName,city,zip,lat,lon,timezone,isp,org,as
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,QLD,Queensland,South Brisbane,4101,-27.4766,153.0166,Australia/Brisbane,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,"AS13335 Cloudflare, Inc."
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,13,Tokyo,Tokyo,270-1351,35.7980,140.1803,Asia/Tokyo,Oracle Corporation,Oracle Cloud Infrastructure (ap-tokyo-1),AS31898 Oracle Corporation
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,ON,Ontario,Toronto,M5H,43.6532,-79.3832,America/Toronto,Windscribe,Zenu Software LLC,AS397540 Windscribe
3,freedns.controld.dev,176.125.239.51,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,ON,Ontario,Toronto,M5H,43.6532,-79.3832,America/Toronto,Windscribe,Zenu Software LLC,AS397540 Windscribe
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,HE,Hesse,Frankfurt am Main,60311,50.1115,8.6810,Europe/Berlin,"nextdns, Inc.","NextDNS, Inc","AS34939 nextdns, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134828,freedns.controld.com,76.76.10.144,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,ON,Ontario,Toronto,M5V 3B1,43.6464,-79.4023,America/Toronto,Controld Inc.,Controld Inc,AS398962 CONTROLD INC.
134829,freedns.controld.com,76.76.10.247,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,ON,Ontario,Toronto,M5V 3B1,43.6464,-79.4023,America/Toronto,Controld Inc.,Controld Inc,AS398962 CONTROLD INC.
134830,premium.dns.controld.com,76.76.2.127,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,ON,Ontario,Toronto,M5V 3B1,43.6464,-79.4023,America/Toronto,Controld Inc.,Controld Inc,AS398962 CONTROLD INC.
134831,dns.controld.com,76.76.2.206,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUd2VE...,DV,True,[Let's Encrypt],True,...,ON,Ontario,Toronto,M5V 3B1,43.6464,-79.4023,America/Toronto,Controld Inc.,Controld Inc,AS398962 CONTROLD INC.


In [14]:
all_raw[all_raw["org"]=="Scape Reach LTD"]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,dane,err,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org


In [10]:
all_raw.drop_duplicates(subset=['ip'], keep='first', inplace=True)
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tlsa_error,caa_error,dnssec,dane,err,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,,,NaN,NaN,NaN,True,True,True,200.0,Strict-Transport-Security: max-age=31536000; i...
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,False,True,True,404.0,NaN
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
3,freedns.controld.dev,176.125.239.51,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,,,NaN,NaN,NaN,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134804,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
134810,dns.iij.jp,210.128.97.73,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjak...,OV,True,[GlobalSign nv-sa],True,...,,,NaN,NaN,NaN,False,False,False,200.0,Strict-Transport-Security: max-age=31536000; i...
134825,dns.nextdns.io,45.90.31.251,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,,,NaN,NaN,NaN,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...
134826,us02.dns4me.net,54.172.110.180,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdiRE...,DV,True,[ZeroSSL],True,...,,,NaN,NaN,NaN,True,True,True,400.0,NaN


In [11]:
all_raw.drop_duplicates(subset=['domain'], keep='first', inplace=True)
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tlsa_error,caa_error,dnssec,dane,err,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,,,NaN,NaN,NaN,True,True,True,200.0,Strict-Transport-Security: max-age=31536000; i...
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,False,True,True,404.0,NaN
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,,,NaN,NaN,NaN,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...
5,dns.sb,185.37.252.149,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdIek...,DV,True,[GoGetSSL],True,...,,,NaN,NaN,NaN,True,False,False,200.0,Strict-Transport-Security: max-age=63072000; i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134691,dns.seey6.xyz,60.205.210.33,,False,read tcp 172.22.21.0:57162->60.205.210.33:443:...,,,False,None,False,...,,,NaN,NaN,NaN,True,True,False,NaN,NaN
134797,dns.nguyendn.com,103.237.147.61,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjVE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,302.0,NaN
134799,doh.dnsfilter.com,103.247.36.150,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN
134804,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,[Let's Encrypt],True,...,,,NaN,NaN,NaN,True,False,False,404.0,NaN


In [12]:
all_raw.keys()

Index(['domain', 'ip', 'tlsVersion', 'scsv', 'connError', 'raw_cert_chain',
       'cert_level', 'certValid', 'caName', 'time_valid', 'cert_lifetime',
       'cert_error', 'tlsHave', 'sct_tls_valid', 'tls_log', 'certHave',
       'cert_valid', 'cert_log', 'ocspHave', 'ocsp_valid', 'ocsp_log',
       'flagValid', 'crLs', 'crl_server', 'ocsp', 'ocspServer', 'ocspStapling',
       'ocspMustStaple', 'respect_ms', 'response_flag', 'crl_cert_status',
       'crl_response_sig', 'ocsp_cert_status', 'ocsp_response_sig',
       'ocsp_stapling_cert_status', 'ocsp_stapling_response_sig', 'caa_result',
       'tlsa', 'caa', 'tlsa_dnssec', 'caa_dnssec', 'tlsa_error', 'caa_error',
       'dnssec', 'dane', 'err', 'tls13_down_12', 'tls13_down_11',
       'tls13_down_10', 'http_code', 'http_headers'],
      dtype='object')

In [13]:
all_raw["tlsa_flag"] = all_raw.apply(lambda x: True if x.tlsa!="" else False, axis=1)
all_raw["caa_flag"] = all_raw.apply(lambda x: True if x.caa!="" else False, axis=1)

In [14]:
all_raw = all_raw.rename(columns={"err":"dane_err"})
# all_raw = all_raw.drop(columns=['rawb_cert_chain','tlsa','caa','tlsa_error','caa_error'])
all_raw = all_raw.drop(columns=['tlsa','caa','tlsa_error','caa_error'])

In [15]:
all_raw = all_raw.fillna("none")

## 去除因域名不匹配导致证书错误的行

In [16]:
all_raw = all_raw.drop(all_raw[all_raw['cert_error'].str.contains("certificate is valid for")].index)
all_raw = all_raw.drop(all_raw[all_raw['cert_error'].str.contains("certificate is not valid for")].index)

In [17]:
all_raw = all_raw[(all_raw["domain"].str.contains("dns"))|(all_raw["domain"].str.contains("doh"))]

In [18]:
all_num = len(all_raw)

# 对TLS进行分析 

## 对错误连接进行分析 

In [16]:
print(all_raw["connError"].value_counts())
print("------------------------------------------------------")
print(all_raw["connError"].value_counts()/all_num)
err = all_raw[all_raw["connError"] != ""]
print("------------------------------------------------------")
print("在{}个DOH解析器中，有{}个连接错误，占比{:.2%}".format(all_num,len(err),len(err)/all_num))

                                                                                   842
dial tcp 130.225.244.166:443: connect: connection refused                            6
dial tcp 149.112.120.3:443: i/o timeout                                              3
dial tcp 137.117.187.53:443: i/o timeout                                             2
dial tcp 157.245.51.195:443: i/o timeout                                             2
dial tcp 208.111.35.106:443: connect: connection refused                             1
read tcp 172.22.21.0:55282->17.253.56.119:443: read: connection reset by peer        1
read tcp 172.22.21.0:39378->82.156.182.7:443: read: connection reset by peer         1
dial tcp 124.82.67.116:443: i/o timeout                                              1
dial tcp 154.85.56.99:443: i/o timeout                                               1
read tcp 172.22.21.0:37682->129.211.209.223:443: read: connection reset by peer      1
dial tcp 185.58.84.198:443: i/o timeout    

In [17]:
err["org"].value_counts()

Danmarks Tekniske Universitet                            6
                                                         5
Host Virtual, Inc                                        4
Apple Inc                                                4
Aliyun Computing Co., LTD                                4
Tencent Cloud Computing (Beijing) Co., Ltd               3
Cira                                                     3
DigitalOcean, LLC                                        2
Huawei Cloud Service                                     2
Microsoft Azure Cloud (westeurope)                       2
Tencent cloud computing (Beijing) Co., Ltd.              2
Mimecast Services Limited                                1
Oracle Cloud Infrastructure (eu-amsterdam-1)             1
Oracle Cloud Infrastructure (ap-seoul-1)                 1
WebHorizon                                               1
OSISM GmbH                                               1
GMO Internet, Inc.                                      

## 对TLS版本进行分析 

In [17]:
all_raw["tlsVersion"].value_counts()

TLSv1.3    750
TLSv1.2     92
            55
Name: tlsVersion, dtype: int64

In [2]:
tls1_3 = all_raw[all_raw["tlsVersion"]=="TLSv1.3"]
tls1_2 = all_raw[all_raw["tlsVersion"]=="TLSv1.2"]
print("在{}个DOT解析器中，有{}个使用TLS1.2进行连接，占比{:.2%}".format(all_num,len(tls1_2),len(tls1_2)/all_num))
print("在{}个DOT解析器中，有{}个使用TLS1.3进行连接，占比{:.2%}".format(all_num,len(tls1_3),len(tls1_3)/all_num))

NameError: name 'all_raw' is not defined

## 对org和ISP进行分析 

In [19]:
print("TLS 1.3")
print("org:")
for index,value in tls1_3["org"].value_counts().iteritems():
    if value >=20:
        print(index,value,"---",value/len(tls1_3)*100) 
print("---------————————————————————————")
print("ISP:")
for index,value in tls1_3["isp"].value_counts().iteritems():
    if value >=20:
        print(index,value,"---",value/len(tls1_3)*100) 

TLS 1.3
org:
 48 --- 6.4
none 29 --- 3.8666666666666667
Hetzner 27 --- 3.5999999999999996
DigitalOcean, LLC 26 --- 3.4666666666666663
---------————————————————————————
ISP:
Oracle Corporation 73 --- 9.733333333333333
Hetzner Online GmbH 34 --- 4.533333333333333
DigitalOcean, LLC 34 --- 4.533333333333333
none 29 --- 3.8666666666666667
OVH SAS 27 --- 3.5999999999999996
Amazon Technologies Inc. 23 --- 3.0666666666666664
Google LLC 22 --- 2.933333333333333
Amazon.com, Inc. 20 --- 2.666666666666667
HostHatch 20 --- 2.666666666666667


In [20]:
print("TLS 1.2")
print("org:")
for index,value in tls1_2["org"].value_counts().iteritems():
    if value >=10:
        print(index,value,"---",value/len(tls1_2)*100) 
print("---------————————————————————————")
print("ISP:")
for index,value in tls1_2["isp"].value_counts().iteritems():
    if value >=10:
        print(index,value,"---",value/len(tls1_2)*100) 

TLS 1.2
org:
 10 --- 10.869565217391305
---------————————————————————————
ISP:


# 对降级攻击进行分析 

In [1]:
scsv = tls1_2[tls1_2["scsv"]==True]
scsv_num = len(scsv)
print("在{}个TLS1.2连接中，有{}个受SCSV保护，占比{:.2%}".format(len(tls1_2),scsv_num,scsv_num/len(tls1_2)))

NameError: name 'tls1_2' is not defined

In [22]:
tls13_down_all = tls1_3[(tls1_3["tls13_down_12"]==True) & (tls1_3["tls13_down_11"]==True) & (tls1_3["tls13_down_10"]==True)]
tls13_down_all_num = len(tls13_down_all)
print("在{}个TLS1.3连接中，有{}个受完全降级攻击保护，占比{:.2%}".format(len(tls1_3),tls13_down_all_num,tls13_down_all_num/len(tls1_3)))

在750个TLS1.3连接中，有78个受完全降级攻击保护，占比10.40%


In [23]:
tls13_down_12 = tls1_3[tls1_3["tls13_down_12"]==True]
tls13_down_12_num = len(tls13_down_12)
print("在{}个TLS1.3连接中，有{}个可以保护降级到TLS 1.2，占比{:.2%}".format(len(tls1_3),tls13_down_12_num,tls13_down_12_num/len(tls1_3)))

在750个TLS1.3连接中，有616个可以保护降级到TLS 1.2，占比82.13%


In [24]:
have_down = all_raw[((all_raw["tlsVersion"]=="TLSv1.2") & (all_raw["scsv"]==True)) | ((all_raw["tlsVersion"]=="TLSv1.3") & (all_raw["tls13_down_12"]==True))]
print("在{}个DOT解析器中，有{}个支持TLS降级保护，占比{:.2%}".format(all_num,len(have_down),len(have_down)/all_num))

在897个DOT解析器中，有686个支持TLS降级保护，占比76.48%


# 对证书进行分析 

## 证书等级 

In [22]:
print(all_raw["cert_level"].value_counts())
print("---------------------------------------")
print(all_raw["cert_level"].value_counts()/all_num*100)

DV                        694
OV                         92
                           55
UnknownValidationLevel     53
EV                          3
Name: cert_level, dtype: int64
---------------------------------------
DV                        77.369008
OV                        10.256410
                           6.131550
UnknownValidationLevel     5.908584
EV                         0.334448
Name: cert_level, dtype: float64


In [26]:
EVcert = all_raw[all_raw["cert_level"]=="EV"]
EVcert

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
432,doh.dns.apple.com,74.80.127.197,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdwVE...,EV,True,[Apple Inc.],True,...,False,False,False,404.0,none,Australia,WoodyNet,WoodyNet,False,False
23307,dns.oryxlabs.com,75.2.118.36,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUlKVE...,EV,True,[Digital Trust L.L.C.],True,...,True,True,True,404.0,none,United States,"Amazon.com, Inc.",AWS Global Accelerator (GLOBAL),False,False
47620,www.dns.oryxlabs.com,75.2.118.36,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUlKVE...,EV,True,[Digital Trust L.L.C.],True,...,False,False,False,404.0,none,United States,"Amazon.com, Inc.",AWS Global Accelerator (GLOBAL),False,False


## 证书有效性 

In [19]:
cert_valid = all_raw[all_raw["certValid"]==True]
cert_invalid = all_raw[all_raw["certValid"]==False]

print(all_raw["certValid"].value_counts())
print("---------------------------------------")
print(all_raw["certValid"].value_counts()/all_num*100)

True     683
False    214
Name: certValid, dtype: int64
---------------------------------------
True     76.142698
False    23.857302
Name: certValid, dtype: float64


In [20]:
cert_valid["org"].value_counts()

                                                             50
none                                                         30
Hetzner                                                      24
DigitalOcean, LLC                                            24
netcup GmbH                                                  14
                                                             ..
Intersect-Host LTD                                            1
DreamScape Networks FZ-LLC                                    1
Technische Universitaet Carolo-Wilhelmina zu Braunschweig     1
Alibaba.com LLC                                               1
Aruba S.p.A. - Cloud Services                                 1
Name: org, Length: 257, dtype: int64

### 根据证书等级分析证书有效性

In [21]:
print("DV:")
print(all_raw[all_raw["cert_level"]=="DV"].certValid.value_counts())
print("---------------------------------------")
print(all_raw[all_raw["cert_level"]=="DV"].certValid.value_counts()/len(all_raw[all_raw["cert_level"]=="DV"])*100)
print("######################################")

print("OV:")
print(all_raw[all_raw["cert_level"]=="OV"].certValid.value_counts())
print("---------------------------------------")
print(all_raw[all_raw["cert_level"]=="OV"].certValid.value_counts()/len(all_raw[all_raw["cert_level"]=="OV"])*100)
print("######################################")

print("EV:")
print(all_raw[all_raw["cert_level"]=="EV"].certValid.value_counts())
print("---------------------------------------")
print(all_raw[all_raw["cert_level"]=="EV"].certValid.value_counts()/len(all_raw[all_raw["cert_level"]=="EV"])*100)

DV:
True     597
False     97
Name: certValid, dtype: int64
---------------------------------------
True     86.023055
False    13.976945
Name: certValid, dtype: float64
######################################
OV:
True     83
False     9
Name: certValid, dtype: int64
---------------------------------------
True     90.217391
False     9.782609
Name: certValid, dtype: float64
######################################
EV:
True    3
Name: certValid, dtype: int64
---------------------------------------
True    100.0
Name: certValid, dtype: float64


### 证书无效错误 

In [30]:
print(all_raw["cert_error"].value_counts())
print("---------------------------------------")
print(all_raw["cert_error"].value_counts()/all_num*100)

                                                                                                                           738
x509: certificate signed by unknown authority                                                                               54
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:24:56+08:00 is after 2021-10-04T23:59:59Z      3
x509: certificate relies on legacy Common Name field, use SANs instead                                                       2
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:20:22+08:00 is after 2020-07-05T22:41:40Z      2
                                                                                                                          ... 
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:21:33+08:00 is after 2015-11-01T00:37:42Z      1
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:21:26+08:00 is after 2021-11-04T0

In [31]:
expired = 0
self_signed = 0
no_san = 0
for index,value in cert_invalid.cert_error.value_counts().iteritems():
    if "expired" in index:
        expired = expired + value
    if "certificate signed by unknown authority" in index:
        self_signed = self_signed + value
    if "use SANs instead" in index:
        no_san = no_san + value
        
print("在{}个错误证书中，有{}个过期，占比{:.2%}".format(len(cert_invalid),expired,expired/len(cert_invalid)))
              
print("在{}个错误证书中，有{}个域名自签名，占比{:.2%}".format(len(cert_invalid),self_signed,self_signed/len(cert_invalid)))

print("在{}个错误证书中，有{}个域名未提供SAN，占比{:.2%}".format(len(cert_invalid),no_san,no_san/len(cert_invalid)))


在214个错误证书中，有103个过期，占比48.13%
在214个错误证书中，有54个域名自签名，占比25.23%
在214个错误证书中，有2个域名未提供SAN，占比0.93%


### 根据证书有效性分析CA

In [32]:
print("证书无效的CA")
for index,value in cert_invalid["caName"].value_counts().iteritems():
     print(index,value,value/len(cert_invalid)*100)

证书无效的CA
["Let's Encrypt"] 76 35.51401869158878
none 55 25.70093457943925
['Mimecast Services Ltd'] 32 14.953271028037381
['Sectigo Limited'] 14 6.5420560747663545
['Virgin Media Ltd.'] 10 4.672897196261682
['DigiCert Inc'] 3 1.4018691588785046
['ZeroSSL'] 3 1.4018691588785046
['TrustAsia Technologies, Inc.'] 3 1.4018691588785046
['SwissSign AG'] 2 0.9345794392523363
['CloudFlare, Inc.'] 2 0.9345794392523363
['TrustOcean Limited'] 1 0.46728971962616817
['vtimothy.com'] 1 0.46728971962616817
['British Telecommunications plc'] 1 0.46728971962616817
['GVC LTD'] 1 0.46728971962616817
['McAfee, Inc.'] 1 0.46728971962616817
['Nagravision'] 1 0.46728971962616817
['Internet Widgits Pty Ltd'] 1 0.46728971962616817
['GeoTrust Inc.'] 1 0.46728971962616817
['Safe Surfer'] 1 0.46728971962616817
['TERENA'] 1 0.46728971962616817
['Superkooka Local'] 1 0.46728971962616817
['COMODO CA Limited'] 1 0.46728971962616817
['GlobalSign nv-sa'] 1 0.46728971962616817
['DoH Private'] 1 0.46728971962616817


In [33]:
print("证书有效的CA")
for index,value in cert_valid["caName"].value_counts().iteritems():
    if value > 10:
        print(index,value,value/len(cert_valid)*100)

证书有效的CA
["Let's Encrypt"] 443 64.86090775988286
['DigiCert Inc'] 56 8.199121522693996
['ZeroSSL'] 53 7.759882869692533
['TrustAsia Technologies, Inc.'] 25 3.6603221083455346
['Sectigo Limited'] 24 3.513909224011713
['GoGetSSL'] 14 2.049780380673499
['Cloudflare, Inc.'] 11 1.610541727672035
['GlobalSign nv-sa'] 11 1.610541727672035


In [34]:
print("在证书无效的情况下，看证书是否过期，false为过期")
print(cert_invalid["time_valid"].value_counts())
print("-----------------------------------------------")
print(cert_invalid["time_valid"].value_counts()/len(cert_invalid))

在证书无效的情况下，看证书是否过期，false为过期
False    155
True      59
Name: time_valid, dtype: int64
-----------------------------------------------
False    0.724299
True     0.275701
Name: time_valid, dtype: float64


#### 分析Let's Encrypt颁发证书的有效性

In [35]:
all_raw[["caName"]] = all_raw[["caName"]].astype(str)
let_encrypt = all_raw[all_raw["caName"]=='["Let\'s Encrypt"]']
print(let_encrypt.certValid.value_counts())
print("---------------------------------------")
print(let_encrypt.certValid.value_counts()/len(let_encrypt)*100)

True     443
False     76
Name: certValid, dtype: int64
---------------------------------------
True     85.356455
False    14.643545
Name: certValid, dtype: float64


In [36]:
let_encrypt.cert_error.value_counts()
expired = 0
self_signed = 0
for index,value in let_encrypt.cert_error.value_counts().iteritems():
    if "expired" in index:
        expired = expired + value
    if "certificate signed by unknown authority" in index:
        self_signed = self_signed + value
        
expired
self_signed

let_encrypt[let_encrypt["cert_error"]=="x509: certificate signed by unknown authority"]

                                                                                                                           443
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:25:47+08:00 is after 2021-07-17T21:38:11Z      2
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:21:27+08:00 is after 2020-07-05T22:41:40Z      2
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:20:22+08:00 is after 2020-07-05T22:41:40Z      2
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:20:38+08:00 is after 2021-11-03T23:29:48Z      2
                                                                                                                          ... 
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:23:14+08:00 is after 2021-08-28T03:04:45Z      1
x509: certificate has expired or is not yet valid: current time 2021-12-21T14:21:33+08:00 is after 2021-09-29T1

76

0

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


## 证书生命周期  

In [37]:
less_91 = 0 
more_91_less_366 = 0
less_398 = 0
more_366_less_731 = 0
more_731 = 0
equal_0 = 0

for index,value in all_raw["cert_lifetime"].value_counts().iteritems():
    if index <= 91 and index > 0:
        less_91 += value
        less_398 += value
    elif index>91 and index<=366:
        more_91_less_366 += value
        less_398 += value
    elif index>366 and index<=398:
        less_398 += value
    elif index>398 and index<=731:
        more_366_less_731 += value
    elif index == 0:
        equal_0 += value
    else:
        more_731 += value
        print(index,value)
        
print("{}个证书生命周期小于398天，占比{:.2%}".format(less_398,less_398/all_num))
print("{}个证书生命周期小于91天，占比{:.2%}".format(less_91,less_91/all_num))
print("{}个证书生命周期大于91天小于等于366天，占比{:.2%}".format(more_91_less_366,more_91_less_366/all_num))
print("{}个证书生命周期大于366天小于等于731天，占比{:.2%}".format(more_366_less_731,more_366_less_731/all_num))
print("{}个证书生命周期大于731天，占比{:.2%}".format(more_731,more_731/all_num))
print("{}个证书生命周期为0，占比{:.2%}".format(equal_0,equal_0/all_num))

5475.0 2
825.0 2
735.5 2
790.4023726851851 1
731.3978009259259 1
1825.0 1
2048.0 1
820.999988425926 1
3650.0 1
731.4027893518519 1
36525.0 1
811个证书生命周期小于398天，占比90.41%
614个证书生命周期小于91天，占比68.45%
125个证书生命周期大于91天小于等于366天，占比13.94%
17个证书生命周期大于366天小于等于731天，占比1.90%
14个证书生命周期大于731天，占比1.56%
55个证书生命周期为0，占比6.13%


In [38]:
all_raw[all_raw["cert_lifetime"]>1095.0].org.value_counts()

DigitalOcean, LLC                            2
Google Cloud                                 1
Vultr Holdings, LLC                          1
Hetzner                                      1
Oracle Cloud Infrastructure (uk-london-1)    1
Name: org, dtype: int64

In [39]:
all_raw[all_raw["cert_lifetime"]>1095.0].caName.value_counts()

['CloudFlare, Inc.']    2
['Safe Surfer']         1
['GVC LTD']             1
['vtimothy.com']        1
['DoH Private']         1
Name: caName, dtype: int64

### 分析org和ISP

In [40]:
print("org:")
for index,value in all_raw[all_raw["cert_lifetime"]<=398.0].org.value_counts().iteritems():
    if value >=30:
        print(index,value,"---",value/len(tls1_3)*100) 
print("---------————————————————————————")
print("ISP:")
for index,value in all_raw[all_raw["cert_lifetime"]<=398.0].org.value_counts().iteritems():
    if value >=30:
        print(index,value,"---",value/len(tls1_3)*100) 

org:
 57 --- 7.6
DigitalOcean, LLC 30 --- 4.0
---------————————————————————————
ISP:
 57 --- 7.6
DigitalOcean, LLC 30 --- 4.0


In [41]:
print("org:")
for index,value in all_raw[all_raw["cert_lifetime"]>=398.0].org.value_counts().iteritems():
    if value >=10:
        print(index,value,"---",value/len(tls1_3)*100) 
print("---------————————————————————————")
print("ISP:")
for index,value in all_raw[all_raw["cert_lifetime"]>=398.0].org.value_counts().iteritems():
    if value >=10:
        print(index,value,"---",value/len(tls1_3)*100) 

org:
---------————————————————————————
ISP:


## 整体分析CA 

In [42]:
for index,value in all_raw["caName"].value_counts().iteritems():
    if value > 20:
        print(index,value,value/all_num*100)

["Let's Encrypt"] 519 57.859531772575245
['DigiCert Inc'] 59 6.577480490523968
['ZeroSSL'] 56 6.243032329988852
none 55 6.131549609810479
['Sectigo Limited'] 38 4.23634336677815
['Mimecast Services Ltd'] 32 3.5674470457079153
['TrustAsia Technologies, Inc.'] 28 3.121516164994426


## 分析证书重用 

In [28]:
temp = all_raw[all_raw["raw_cert_chain"]!=""]
len(temp)

842

In [29]:
cert_share_list = [] 
share_domain = 0
shart_cert = 0
for index,value in temp["raw_cert_chain"].value_counts().iteritems():
    if value > 10:
        print(value,value/all_num*100)
        cert_share_list.append(index)
        shart_cert = shart_cert + 1
        share_domain = share_domain + value
    elif value > 1:
        shart_cert = shart_cert + 1
        share_domain = share_domain + value
# cert_share_list
print("一共有{}域名共享证书，占比{:.2%}".format(share_domain,share_domain/len(temp)))
print("一共有{}证书被共享，占比{:.2%}".format(shart_cert,shart_cert/len(temp["raw_cert_chain"].value_counts())))

11 1.2263099219620959
一共有274域名共享证书，占比32.54%
一共有95证书被共享，占比14.33%


### 对重用比例较高的证书进行分析 

#### 1 

In [30]:
cert_share_1 = all_raw[all_raw["raw_cert_chain"]==cert_share_list[0]]
cert_share_1["org"].value_counts()
cert_share_1["isp"].value_counts()

nextdns, Inc.                    4
DigitalOcean, LLC                2
NextDNS, Inc                     1
tavu.io Public Cloud Services    1
Nextdns Sin3                     1
Nextdns Hkg1                     1
Nextdns Tyo1                     1
Name: org, dtype: int64

nextdns              4
nextdns, Inc.        4
DigitalOcean, LLC    2
Multim Oy            1
Name: isp, dtype: int64

In [31]:
cert_share_1["certValid"].value_counts()

True    11
Name: certValid, dtype: int64

In [27]:
cert_share_1["raw_cert_chain"]

530       
4668      
4925      
4929      
4964      
4971      
5021      
5022      
5236      
5274      
22702     
22936     
23069     
23314     
47507     
47508     
47782     
47798     
52406     
68452     
68478     
68533     
84199     
84529     
84673     
84944     
90728     
96226     
96710     
96761     
96828     
98256     
98891     
99126     
119998    
120096    
120602    
120873    
121622    
121660    
123256    
123257    
128706    
128967    
129012    
129169    
129495    
129497    
129524    
129552    
129811    
134368    
134398    
134534    
134691    
Name: raw_cert_chain, dtype: object

In [32]:
cert_share_1[cert_share_1["certValid"]==False].cert_error

Series([], Name: cert_error, dtype: object)

In [33]:
cert_share_1[cert_share_1["certValid"]==False]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


#### 2 

In [34]:
cert_share_1 = all_raw[all_raw["raw_cert_chain"]==cert_share_list[1]]
cert_share_1["org"].value_counts()
cert_share_1["isp"].value_counts()

IndexError: list index out of range

In [49]:
cert_share_1["certValid"].value_counts()

True    11
Name: certValid, dtype: int64

In [50]:
cert_share_1[cert_share_1["certValid"]==False].cert_error.value_counts()

Series([], Name: cert_error, dtype: int64)

# 对CT进行分析 

In [51]:
def list_convert(arg):
    new_list = []
    for i in arg:
        if type(i) is not list:
            new_list.append(i)
        else:
            new_list.extend(list_convert(i))
    return new_list

## 分析sct传递方式

### 拥有SCT的任意一种传递方式的DOH个数（至少包含三种方式的其中一种） 

In [52]:
have_sct = all_raw.query('tlsHave == True | certHave == True | ocspHave == True')
have_sct_num = have_sct.ip.count()
print('{}个域名有SCT，在全部DOT中占比{:.2%}'.format(have_sct_num, have_sct_num/len(all_raw)))

789个域名有SCT，在全部DOT中占比87.96%


#### 拥有SCT的全部传递方式的DOT个数

In [53]:
have_sct_all = all_raw.query('tlsHave == True & certHave == True & ocspHave == True').ip.count()
print('{}个域名含有全部SCT的传递方式，占比{:.2%}'.format(have_sct_all, have_sct_all/len(all_raw)))

0个域名含有全部SCT的传递方式，占比0.00%


#### 通过tls传播DOT的域名个数 

In [54]:
sct_tls = all_raw[all_raw['tlsHave'] == True]
sct_tls_num = sct_tls.ip.count()
print('{}个域名通过tls传递SCT，占比{:.4%}'.format(sct_tls_num, sct_tls_num/len(all_raw)))
print("在支持CT的DOT中占比：{:.2%}".format(sct_tls_num/have_sct_num))

0个域名通过tls传递SCT，占比0.0000%
在支持CT的DOT中占比：0.00%


#### 通过cert传播DOT的域名个数 

In [55]:
sct_cert = all_raw[all_raw['certHave'] == True]
sct_cert_num = sct_cert.ip.count()
print('{}个域名通过cert传递SCT，占比{:.4%}'.format(sct_cert_num, sct_cert_num/len(all_raw)))
print("在支持CT的DOT中占比：{:.2%}".format(sct_cert_num/have_sct_num))

789个域名通过cert传递SCT，占比87.9599%
在支持CT的DOT中占比：100.00%


#### 通过ocsp传播DOT的域名个数 

In [56]:
sct_ocsp = all_raw[all_raw['ocspHave'] == True]
sct_ocsp_num = sct_ocsp.ip.count()
print('{}个域名通过ocsp传递SCT，占比{:.4%}'.format(sct_ocsp_num, sct_ocsp_num/len(all_raw)))
print("在支持CT的DOT中占比：{:.2%}".format(sct_ocsp_num/have_sct_num))

5个域名通过ocsp传递SCT，占比0.5574%
在支持CT的DOT中占比：0.63%


In [57]:
sct_ocsp

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,Germany,"nextdns, Inc.","NextDNS, Inc",False,False
14,dns1.nextdns.io,45.90.28.230,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVKak...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,"nextdns, Inc.","nextdns, Inc",False,False
15,dns2.nextdns.io,45.90.30.122,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVLVE...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,"nextdns, Inc.","nextdns, Inc",False,False
568,chromium.dns.nextdns.io,45.11.104.186,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,['ZeroSSL'],True,...,True,False,False,none,none,Hong Kong,Misaka Network,"nextdns, Inc",False,False
97180,ordns-west.2wi.nl,133.18.198.219,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUhsek...,DV,True,['ZeroSSL'],True,...,True,False,False,401.0,none,Japan,KAGOYA JAPAN Inc.,,False,False


## 分析拥有CT的CA 

In [58]:
print(have_sct["caName"].value_counts())
print("-------------------------------")
print(have_sct["caName"].value_counts()/len(have_sct)*100)

["Let's Encrypt"]                                                   519
['DigiCert Inc']                                                     59
['ZeroSSL']                                                          56
['Sectigo Limited']                                                  38
['TrustAsia Technologies, Inc.']                                     28
['GoGetSSL']                                                         14
['GlobalSign nv-sa']                                                 12
['Cloudflare, Inc.']                                                 11
['GoDaddy.com, Inc.']                                                 6
['Google Trust Services LLC']                                         6
['Buypass AS-983163327']                                              5
['GEANT Vereniging']                                                  5
['TrustOcean Limited']                                                3
['Verein zur Foerderung eines Deutschen Forschungsnetzes e. V.']

## 分析sct有效性 

### 至少有一个sct验证成功

In [59]:
sct_valid = all_raw[all_raw['flagValid'] == True]
print("在所有DOT中，至少有一个sct有效的DOT：")
print(all_raw['flagValid'].value_counts())
print("-------------------------------")
print(all_raw['flagValid'].value_counts()/len(all_raw)*100)

print("**************************************")

print("在支持CT的DOT中：")
print(have_sct['flagValid'].value_counts())
print("-------------------------------")
print(have_sct['flagValid'].value_counts()/len(have_sct)*100)

在所有DOT中，至少有一个sct有效的DOT：
True     761
False    136
Name: flagValid, dtype: int64
-------------------------------
True     84.83835
False    15.16165
Name: flagValid, dtype: float64
**************************************
在支持CT的DOT中：
True     761
False     28
Name: flagValid, dtype: int64
-------------------------------
True     96.451204
False     3.548796
Name: flagValid, dtype: float64


### 分析通过cert传递的sct的验证情况 

In [60]:
cert_valid_list = []
for index,value in sct_cert["cert_valid"].iteritems():
    temp = value.split("#||#")
    cert_valid_list.append(temp)
    
temp = list_convert(cert_valid_list)
sct_cert_verify =pd.Series(temp)
print("通过cert传递的所有sct的验证情况：")
print(sct_cert_verify.value_counts())
print("-------------------------------")
print(sct_cert_verify.value_counts()/len(temp)*100)

通过cert传递的所有sct的验证情况：
true     1563
false     180
dtype: int64
-------------------------------
true     89.672978
false    10.327022
dtype: float64


### 分析通过ocsp传递的sct的验证情况 

In [61]:
ocsp_valid_list = []
for index,value in sct_ocsp["cert_valid"].iteritems():
    temp = value.split("#||#")
    ocsp_valid_list.append(temp)
    
temp = list_convert(ocsp_valid_list)
sct_ocsp_verify =pd.Series(temp)
print("通过ocsp传递的所有sct的验证情况：")
print(sct_ocsp_verify.value_counts())
print("-------------------------------")
print(sct_ocsp_verify.value_counts()/len(temp)*100)

通过ocsp传递的所有sct的验证情况：
true    10
dtype: int64
-------------------------------
true    100.0
dtype: float64


## 分析sct的logdescription 

### 分析通过cert传递的sct的log 

In [62]:
cert_log_list = []
for index,value in sct_cert["cert_log"].iteritems():
    temp = value.split("#||#")
    cert_log_list.append(temp)
    
temp = list_convert(cert_log_list)
cert_log =pd.Series(temp)
print(cert_log.value_counts())   #null即该域名有cert传递的sct，但是certvalid都为false。或第一个为flase,第二个为true
print("-------------------------------")
print(cert_log.value_counts()/len(temp)*100)

Google 'Argon2022' log         336
Google 'Xenon2022' log         331
Cloudflare 'Nimbus2022' Log    298
Let's Encrypt 'Oak2022' log    188
                                78
DigiCert Nessie2022 Log         78
Sectigo 'Mammoth' CT log        74
Google 'Xenon2021' log          63
Google 'Argon2021' log          52
Cloudflare 'Nimbus2021' Log     43
Let's Encrypt 'Oak2021' log     36
DigiCert Yeti2021 Log           32
Sectigo 'Sabre' CT log          13
Google 'Skydiver' log            5
Google 'Rocketeer' log           5
Google 'Pilot' log               4
DigiCert Log Server              4
Trust Asia Log2022               1
dtype: int64
-------------------------------
Google 'Argon2022' log         20.475320
Google 'Xenon2022' log         20.170628
Cloudflare 'Nimbus2022' Log    18.159659
Let's Encrypt 'Oak2022' log    11.456429
                                4.753199
DigiCert Nessie2022 Log         4.753199
Sectigo 'Mammoth' CT log        4.509445
Google 'Xenon2021' log          3.8391

### 分析通过ocsp传递的sct的log 

In [63]:
ocsp_log_list = []
for index,value in sct_ocsp["cert_log"].iteritems():
    temp = value.split("#||#")
    ocsp_log_list.append(temp)
    
temp = list_convert(ocsp_log_list)
ocsp_log =pd.Series(temp)
print(ocsp_log.value_counts())   #null即该域名有cert传递的sct，但是certvalid都为false。或第一个为flase,第二个为true
print("-------------------------------")
print(ocsp_log.value_counts()/len(temp)*100)

Google 'Xenon2022' log         5
Cloudflare 'Nimbus2022' Log    4
Let's Encrypt 'Oak2022' log    1
dtype: int64
-------------------------------
Google 'Xenon2022' log         50.0
Cloudflare 'Nimbus2022' Log    40.0
Let's Encrypt 'Oak2022' log    10.0
dtype: float64


### 分析整体的log

In [64]:
sct_log = pd.concat([cert_log, ocsp_log], axis=0,ignore_index=True)
print(sct_log.value_counts())
print("-------------------------------")
print(sct_log.value_counts()/len(sct_log)*100)

Google 'Argon2022' log         336
Google 'Xenon2022' log         336
Cloudflare 'Nimbus2022' Log    302
Let's Encrypt 'Oak2022' log    189
                                78
DigiCert Nessie2022 Log         78
Sectigo 'Mammoth' CT log        74
Google 'Xenon2021' log          63
Google 'Argon2021' log          52
Cloudflare 'Nimbus2021' Log     43
Let's Encrypt 'Oak2021' log     36
DigiCert Yeti2021 Log           32
Sectigo 'Sabre' CT log          13
Google 'Skydiver' log            5
Google 'Rocketeer' log           5
Google 'Pilot' log               4
DigiCert Log Server              4
Trust Asia Log2022               1
dtype: int64
-------------------------------
Google 'Argon2022' log         20.351302
Google 'Xenon2022' log         20.351302
Cloudflare 'Nimbus2022' Log    18.291944
Let's Encrypt 'Oak2022' log    11.447608
                                4.724409
DigiCert Nessie2022 Log         4.724409
Sectigo 'Mammoth' CT log        4.482132
Google 'Xenon2021' log          3.8158

### 整体分析log的多样性

In [65]:
log_list = []
for index,value in have_sct["cert_log"].iteritems():
    temp = value.split("#||#")
    log_list.append(temp)

for index,value in have_sct["ocsp_log"].iteritems():
    temp = value.split("#||#")
    log_list.append(temp)

log_num = len(log_list)    

log_more = 0
log_one = 0
one_google = 0
log_nan = 0
for item in log_list:
    if type(item) == list:
        uniq_item = list(set(item))  ##去重
        if len(uniq_item) >1:
            log_more += 1
        elif len(uniq_item) == 1:
            log_one += 1
            if "Google" in str(uniq_item[0]):
                one_google += 1
    else:
        log_nan += 1
            

print("在{}个sct的log中:".format(log_num))
print("有{}个sct有超过一个的log操作员,占比{:.2%}".format(log_more,log_more/log_num))
print("有{}个sct只有一个的log操作员,占比{:.2%}。其中google有{}个，占比{:.2%}".format(log_one,log_one/log_num,one_google,one_google/log_one))
print("剩余{}个没有获取到log操作员，占比{:.2%}".format(log_nan,log_nan/log_num))

在1578个sct的log中:
有715个sct有超过一个的log操作员,占比45.31%
有863个sct只有一个的log操作员,占比54.69%。其中google有51个，占比5.91%
剩余0个没有获取到log操作员，占比0.00%


## 分析不支持CT的域名

### 整体分析没有sct的caname

In [66]:
no_sct = all_raw.query('tlsHave == False & certHave == False & ocspHave == False')
print("没有sct的域名共有：",no_sct.ip.count())
print(no_sct["caName"].value_counts())

没有sct的域名共有： 108
none                                  55
['Mimecast Services Ltd']             32
['Virgin Media Ltd.']                 10
['CloudFlare, Inc.']                   2
['Superkooka Local']                   1
['Safe Surfer']                        1
['Nagravision']                        1
['GeoTrust Inc.']                      1
['Internet Widgits Pty Ltd']           1
['GVC LTD']                            1
['British Telecommunications plc']     1
['vtimothy.com']                       1
['DoH Private']                        1
Name: caName, dtype: int64


In [67]:
 for i in range(len(no_sct)):
        if no_sct.iloc[i]["caName"] != "none":
            for value in no_sct.iloc[i]["caName"]:
                if "Fortinet" in value:
                    print(no_sct.iloc[i]["domain"],no_sct.iloc[i]["ip"])

### 在证书有效的情况下，分析sct无效的域名 

In [68]:
%%time
Cert_valid_sctfalse = cert_valid[cert_valid['flagValid'] == False]
Cert_valid_sctfalse_num = Cert_valid_sctfalse.ip.count()
print("证书有效但sct无效的域名共有：",Cert_valid_sctfalse_num)
Cert_valid_sctfalse

证书有效但sct无效的域名共有： 7
Wall time: 1.99 ms


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
22637,dns.wevpn.com,72.11.134.90,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUhMRE...,DV,True,[Sectigo Limited],True,...,False,False,False,none,none,United States,QuadraNet Enterprises LLC,QuadraNet Enterprises LLC,False,False
23291,dns.csaonline.de,213.166.247.100,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZZVE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Germany,FAN Media GmbH,,False,False
68263,dns.csa-rz.de,81.27.162.100,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZZVE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Germany,R-KOM Regensburger Telekommunikationsgesellsch...,,False,False
68573,tlsdns.xuanlove.cn,1.117.174.35,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdFRE...,DV,True,"[TrustAsia Technologies, Inc.]",True,...,True,False,False,302.0,none,China,China Internet Network Information Center,"Tencent cloud computing (Beijing) Co., Ltd.",False,False
90965,powerdns-test.csaonline.de,81.27.162.100,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZZVE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Germany,R-KOM Regensburger Telekommunikationsgesellsch...,,False,False
98920,doh.csaonline.de,81.27.162.100,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZZVE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Germany,R-KOM Regensburger Telekommunikationsgesellsch...,,False,False
99057,dns.leonardo.tec.br,150.136.0.232,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdTVE...,DV,True,[Sectigo Limited],True,...,True,False,False,302.0,none,United States,Oracle Corporation,Oracle Cloud Infrastructure (us-ashburn-1),False,False


In [69]:
Cert_valid_sctfalse["caName"]

22637                 [Sectigo Limited]
23291                   [Let's Encrypt]
68263                   [Let's Encrypt]
68573    [TrustAsia Technologies, Inc.]
90965                   [Let's Encrypt]
98920                   [Let's Encrypt]
99057                 [Sectigo Limited]
Name: caName, dtype: object

## 分析EV证书的CT

In [70]:
EVcert

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
432,doh.dns.apple.com,74.80.127.197,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdwVE...,EV,True,[Apple Inc.],True,...,False,False,False,404.0,none,Australia,WoodyNet,WoodyNet,False,False
23307,dns.oryxlabs.com,75.2.118.36,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUlKVE...,EV,True,[Digital Trust L.L.C.],True,...,True,True,True,404.0,none,United States,"Amazon.com, Inc.",AWS Global Accelerator (GLOBAL),False,False
47620,www.dns.oryxlabs.com,75.2.118.36,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUlKVE...,EV,True,[Digital Trust L.L.C.],True,...,False,False,False,404.0,none,United States,"Amazon.com, Inc.",AWS Global Accelerator (GLOBAL),False,False


In [71]:
EVcert["flagValid"]
EVcert["caName"]
EVcert["cert_log"]

432      True
23307    True
47620    True
Name: flagValid, dtype: bool

432                [Apple Inc.]
23307    [Digital Trust L.L.C.]
47620    [Digital Trust L.L.C.]
Name: caName, dtype: object

432      Google 'Pilot' log#||#DigiCert Log Server#||#G...
23307     Google 'Xenon2022' log#||#Sectigo 'Sabre' CT log
47620    Google 'Xenon2022' log#||#Sectigo 'Sabre' CT l...
Name: cert_log, dtype: object

# 对证书撤销进行分析 

In [72]:
all_raw.keys()

Index(['domain', 'ip', 'tlsVersion', 'scsv', 'connError', 'raw_cert_chain',
       'cert_level', 'certValid', 'caName', 'time_valid', 'cert_lifetime',
       'cert_error', 'tlsHave', 'sct_tls_valid', 'tls_log', 'certHave',
       'cert_valid', 'cert_log', 'ocspHave', 'ocsp_valid', 'ocsp_log',
       'flagValid', 'crLs', 'crl_server', 'ocsp', 'ocspServer', 'ocspStapling',
       'ocspMustStaple', 'respect_ms', 'response_flag', 'crl_cert_status',
       'crl_response_sig', 'ocsp_cert_status', 'ocsp_response_sig',
       'ocsp_stapling_cert_status', 'ocsp_stapling_response_sig', 'caa_result',
       'tlsa_dnssec', 'caa_dnssec', 'dnssec', 'dane', 'dane_err',
       'tls13_down_12', 'tls13_down_11', 'tls13_down_10', 'http_code',
       'http_headers', 'country', 'isp', 'org', 'tlsa_flag', 'caa_flag'],
      dtype='object')

## 分析证书撤销方式 

### 整体分析 

In [73]:
have_crl = all_raw[all_raw["crLs"] == True]
have_ocsp = all_raw[all_raw["ocsp"] == True]
have_ocspsl = all_raw[all_raw["ocspStapling"] == True]
have_ocspms = all_raw[all_raw["ocspMustStaple"] == True]
print("在所有DOT中：")
print("CRL传递方式")
print(all_raw["crLs"].value_counts())
print("-------------------------------")
print(all_raw["crLs"].value_counts()/all_num*100)
print("**************************************")
print("OCSP传递方式")
print(all_raw["ocsp"].value_counts())
print("-------------------------------")
print(all_raw["ocsp"].value_counts()/all_num*100)
print("**************************************")
print("ocspStapling传递方式")
print(all_raw["ocspStapling"].value_counts())
print("-------------------------------")
print(all_raw["ocspStapling"].value_counts()/all_num*100)
print("**************************************")
print("ocspMustStaple传递方式")
print(all_raw["ocspMustStaple"].value_counts())
print("-------------------------------")
print(all_raw["ocspMustStaple"].value_counts()/all_num*100)

在所有DOT中：
CRL传递方式
False    767
True     130
Name: crLs, dtype: int64
-------------------------------
False    85.507246
True     14.492754
Name: crLs, dtype: float64
**************************************
OCSP传递方式
True     791
False    106
Name: ocsp, dtype: int64
-------------------------------
True     88.182832
False    11.817168
Name: ocsp, dtype: float64
**************************************
ocspStapling传递方式
False    695
True     202
Name: ocspStapling, dtype: int64
-------------------------------
False    77.480491
True     22.519509
Name: ocspStapling, dtype: float64
**************************************
ocspMustStaple传递方式
False    871
True      26
Name: ocspMustStaple, dtype: int64
-------------------------------
False    97.101449
True      2.898551
Name: ocspMustStaple, dtype: float64


### 分析支持证书撤销的DOT 

In [74]:
have_revoke = all_raw[(all_raw["crLs"]==True) | (all_raw["ocsp"]==True) | (all_raw["ocspStapling"]==True) | (all_raw["ocspMustStaple"]==True)]
print("在所有DOT中，支持证书撤销的有：{}, 占比{:.2%}".format(len(have_revoke), len(have_revoke)/all_num))
have_revoke

在所有DOT中，支持证书撤销的有：793, 占比88.41%


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,['DigiCert Inc'],True,...,True,True,True,200.0,Strict-Transport-Security: max-age=31536000; i...,Australia,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,False,False
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,"[""Let's Encrypt""]",True,...,False,True,True,404.0,none,Japan,Oracle Corporation,Oracle Cloud Infrastructure (ap-tokyo-1),False,False
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,Canada,Windscribe,Zenu Software LLC,False,False
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,Germany,"nextdns, Inc.","NextDNS, Inc",False,False
5,dns.sb,185.37.252.149,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdIek...,DV,True,['GoGetSSL'],True,...,True,False,False,200.0,Strict-Transport-Security: max-age=63072000; i...,Estonia,xTom GmbH,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134797,dns.nguyendn.com,103.237.147.61,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjVE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,302.0,none,Vietnam,LVSOFT,,False,False
134799,doh.dnsfilter.com,103.247.36.150,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,United States,DNSFilter,"DNSFilter, Inc.",False,False
134802,cv-dns-on-ca-3.cloudveil.org,142.93.146.238,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoak...,DV,False,['Sectigo Limited'],False,...,True,True,True,404.0,none,Canada,"DigitalOcean, LLC","DigitalOcean, LLC",False,False
134804,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,Poland,Aruba S.p.A.,Aruba S.p.A. - Cloud Services,False,False


### 只有CRL的情况

In [75]:
only_crl = all_raw[(all_raw["crLs"]==True) & (all_raw["ocsp"]==False) & (all_raw["ocspStapling"]==False) & (all_raw["ocspMustStaple"]==False)]
print("在所有DOT中，只支持CRL的有：{}, 占比{:.2%}".format(len(only_crl), len(only_crl)/all_num))
only_crl

在所有DOT中，只支持CRL的有：2, 占比0.22%


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
84805,dns.2l2o.com,144.202.31.179,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVsRE...,UnknownValidationLevel,False,"['CloudFlare, Inc.']",True,...,False,False,False,301.0,Strict-Transport-Security: max-age=63072000,United States,The Constant Company,"Vultr Holdings, LLC",False,False
129269,dns.nephren.org,150.230.116.189,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVsek...,UnknownValidationLevel,False,"['CloudFlare, Inc.']",True,...,False,False,False,302.0,none,United Kingdom,Oracle Corporation,Oracle Cloud Infrastructure (uk-london-1),False,False


### 只有CRL或OCSP的情况

In [76]:
only_crl_ocsp = all_raw[((all_raw["crLs"]==True) | (all_raw["ocsp"]==True)) & (all_raw["ocspStapling"]==False) & (all_raw["ocspMustStaple"]==False)]
print("在所有DOT中，只支持CRL或OCSP的有：{}, 占比{:.2%}".format(len(only_crl_ocsp), len(only_crl_ocsp)/all_num))
only_crl_ocsp

在所有DOT中，只支持CRL或OCSP的有：590, 占比65.77%


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,"[""Let's Encrypt""]",True,...,False,True,True,404.0,none,Japan,Oracle Corporation,Oracle Cloud Infrastructure (ap-tokyo-1),False,False
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,Canada,Windscribe,Zenu Software LLC,False,False
18,rdns.faelix.net,46.227.204.54,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKVE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,400.0,none,United Kingdom,Faelix Limited,Faelix (TE ),False,False
19,resolver4.dns.openinternet.io,70.36.170.126,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZQek...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,United States,Sonic.net,Private Customer - Sonic.net,False,False
20,doh.xfinity.com,75.75.77.99,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUhCRE...,OV,True,['COMODO CA Limited'],True,...,True,True,True,400.0,none,United States,Comcast Cable Communications,"Comcast Cable Communications Holdings, Inc",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134797,dns.nguyendn.com,103.237.147.61,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjVE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,302.0,none,Vietnam,LVSOFT,,False,False
134799,doh.dnsfilter.com,103.247.36.150,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,United States,DNSFilter,"DNSFilter, Inc.",False,False
134802,cv-dns-on-ca-3.cloudveil.org,142.93.146.238,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoak...,DV,False,['Sectigo Limited'],False,...,True,True,True,404.0,none,Canada,"DigitalOcean, LLC","DigitalOcean, LLC",False,False
134804,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,404.0,none,Poland,Aruba S.p.A.,Aruba S.p.A. - Cloud Services,False,False


### 分析OCSP Must-Staple 

In [77]:
print("在支持ocspMustStaple的DOT中，提供OCSPStapling的有：")
print(have_ocspms["respect_ms"].value_counts())
print("-------------------------------")
print(have_ocspms["respect_ms"].value_counts()/len(have_ocspms)*100)

在支持ocspMustStaple的DOT中，提供OCSPStapling的有：
True     25
False     1
Name: respect_ms, dtype: int64
-------------------------------
True     96.153846
False     3.846154
Name: respect_ms, dtype: float64


In [78]:
print("org")
have_ocspms["org"].value_counts()
print("isp")
have_ocspms["isp"].value_counts()

org


nextdns, Inc.                    4
Hetzner                          4
nextdns, Inc                     3
none                             3
NextDNS, Inc                     2
DigitalOcean, LLC                2
                                 2
tavu.io Public Cloud Services    1
Nextdns Sin3                     1
HETZNER-DC                       1
Nextdns Hkg1                     1
Fasthosts Internet Limited       1
Nextdns Tyo1                     1
Name: org, dtype: int64

isp


nextdns, Inc.          7
Hetzner Online GmbH    5
nextdns                4
none                   3
DigitalOcean, LLC      2
Misaka Network         1
Multim Oy              1
KAGOYA JAPAN Inc.      1
Triolan                1
IONOS SE               1
Name: isp, dtype: int64

#### 在有OCSP Must-Staple的情况下是否提供OCSP Stapling

In [79]:
respect_ms = have_ocspms[have_ocspms["respect_ms"]==True]
respect_ms

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,Germany,"nextdns, Inc.","NextDNS, Inc",False,False
10,lax-1.edge.nextdns.io,23.247.214.45,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,"[""Let's Encrypt""]",True,...,False,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,France,nextdns,"nextdns, Inc.",False,False
12,dal-1.edge.nextdns.io,23.247.215.95,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,"[""Let's Encrypt""]",True,...,False,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,nextdns,"nextdns, Inc.",False,False
14,dns1.nextdns.io,45.90.28.230,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVKak...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,"nextdns, Inc.","nextdns, Inc",False,False
15,dns2.nextdns.io,45.90.30.122,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVLVE...,DV,True,['ZeroSSL'],True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,"nextdns, Inc.","nextdns, Inc",False,False
351,ams-1.edge.nextdns.io,185.253.5.115,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,"[""Let's Encrypt""]",True,...,False,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,Germany,"nextdns, Inc.","NextDNS, Inc",False,False
355,ewr-1.edge.nextdns.io,23.247.212.126,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,"[""Let's Encrypt""]",True,...,False,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,nextdns,"nextdns, Inc.",False,False
563,chi-1.edge.nextdns.io,23.247.213.204,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,"[""Let's Encrypt""]",True,...,False,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,nextdns,"nextdns, Inc.",False,False
568,chromium.dns.nextdns.io,45.11.104.186,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,['ZeroSSL'],True,...,True,False,False,none,none,Hong Kong,Misaka Network,"nextdns, Inc",False,False
68472,do-sfo-1.edge.nextdns.io,143.110.229.87,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,"[""Let's Encrypt""]",True,...,True,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,United States,"DigitalOcean, LLC","DigitalOcean, LLC",False,False


In [80]:
respect_no_ms = have_ocspms[have_ocspms["respect_ms"]==False]
respect_no_ms

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
119966,doh.cosmonova.club,159.224.47.130,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVOek...,DV,True,['ZeroSSL'],True,...,True,False,False,404.0,none,Ukraine,Triolan,,False,False


#### 支持ocsp must-staple，不提供ocspstapling的org和ISP 

In [81]:
print("org")
respect_no_ms["org"].value_counts()
print("isp")
respect_no_ms["isp"].value_counts()

org


    1
Name: org, dtype: int64

isp


Triolan    1
Name: isp, dtype: int64

### 同时支持四种传递方式

In [82]:
all_revoke = all_raw[(all_raw["crLs"]==True) & (all_raw["ocsp"]==True) & (all_raw["ocspStapling"]==True) & (all_raw["ocspMustStaple"]==True)]
print("在所有DOH中，同时支持四种传递方式的有：{}, 占比{:.2%}".format(len(all_revoke), len(all_revoke)/all_num))
all_revoke

在所有DOH中，同时支持四种传递方式的有：0, 占比0.00%


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


## 从证书撤销角度分析CA 

In [83]:
print("CRL:")
for index,value in have_crl["caName"].value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_crl)*100) 

print("-------------------")

print("OCSP:")
for index,value in have_ocsp["caName"].value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_ocsp)*100) 

print("-------------------")

print("OCSPStapling:")
for index,value in have_ocspsl["caName"].value_counts().iteritems():
    print(index,value,value/len(have_ocspsl)*100) 

print("-------------------")

print("OCSP Must-Staple:")
for index,value in have_ocspms["caName"].value_counts().iteritems():
    print(index,value,value/len(have_ocspms)*100) 

CRL:
['DigiCert Inc'] 46 35.38461538461539
['GoGetSSL'] 14 10.76923076923077
['Cloudflare, Inc.'] 11 8.461538461538462
-------------------
OCSP:
["Let's Encrypt"] 519 65.61314791403287
['DigiCert Inc'] 59 7.458912768647282
['ZeroSSL'] 56 7.079646017699115
['Sectigo Limited'] 38 4.804045512010114
['TrustAsia Technologies, Inc.'] 28 3.5398230088495577
['GoGetSSL'] 14 1.7699115044247788
['GlobalSign nv-sa'] 12 1.5170670037926675
['Cloudflare, Inc.'] 11 1.390644753476612
-------------------
OCSPStapling:
["Let's Encrypt"] 131 64.85148514851485
['DigiCert Inc'] 24 11.881188118811881
['ZeroSSL'] 13 6.435643564356436
['GoGetSSL'] 12 5.9405940594059405
['Cloudflare, Inc.'] 11 5.445544554455446
['GEANT Vereniging'] 5 2.4752475247524752
['TrustAsia Technologies, Inc.'] 2 0.9900990099009901
['Sectigo Limited'] 2 0.9900990099009901
['Apple Inc.'] 1 0.49504950495049505
['British Telecommunications plc'] 1 0.49504950495049505
-------------------
OCSP Must-Staple:
["Let's Encrypt"] 20 76.923076923076

## 从证书撤销角度分析org

In [84]:
print("CRL:")
for index,value in have_crl["org"].value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_crl)*100) 

print("-------------------")

print("OCSP:")
for index,value in have_ocsp["org"].value_counts().iteritems():
    if value >= 20:
        print(index,value,value/len(have_ocsp)*100) 

print("-------------------")

print("OCSPStapling:")
for index,value in have_ocspsl["org"].value_counts().iteritems():
    print(index,value,value/len(have_ocspsl)*100) 

print("-------------------")

print("OCSP Must-Staple:")
for index,value in have_ocspms["org"].value_counts().iteritems():
    print(index,value,value/len(have_ocspms)*100) 

CRL:
none 17 13.076923076923078
 10 7.6923076923076925
-------------------
OCSP:
 56 7.079646017699115
none 30 3.792667509481669
DigitalOcean, LLC 29 3.6662452591656134
Hetzner 27 3.4134007585335016
-------------------
OCSPStapling:
none 19 9.405940594059405
 18 8.91089108910891
DigitalOcean, LLC 10 4.9504950495049505
HostHatch LLC 9 4.455445544554455
HostHatch, LLC 8 3.9603960396039604
Cisco OpenDNS LLC 7 3.4653465346534658
Hetzner 7 3.4653465346534658
APNIC and Cloudflare DNS Resolver project 5 2.4752475247524752
Packet Clearing House 5 2.4752475247524752
Danmarks Tekniske Universitet 5 2.4752475247524752
WEBPARTNER 4 1.9801980198019802
Digital Ocean 4 1.9801980198019802
Kracon ApS 4 1.9801980198019802
nextdns, Inc. 4 1.9801980198019802
Institute of Mathematics and Computer Science of University of Latvia 4 1.9801980198019802
OVH Australia PTY LTD 3 1.4851485148514851
Thomas Steen Rasmussen 3 1.4851485148514851
Patrick Velder 3 1.4851485148514851
Dolphin IT-Systeme e.K. 3 1.485148514

## 分析CRL 和 OCSP Server 

In [85]:
print("CRL Server")
for index,value in have_crl["crl_server"].value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_crl)*100) 
print("------------------------------------------------------")
print("OCSP Server：")
for index,value in have_ocsp["ocspServer"].value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_ocsp)*100) 

CRL Server
['http://crl3.digicert.com/DigiCertTLSRSASHA2562020CA1.crl', 'http://crl4.digicert.com/DigiCertTLSRSASHA2562020CA1.crl'] 16 12.307692307692308
['http://crl.usertrust.com/GoGetSSLRSADVCA.crl'] 14 10.76923076923077
['http://crl3.digicert.com/CloudflareIncECCCA-3.crl', 'http://crl4.digicert.com/CloudflareIncECCCA-3.crl'] 11 8.461538461538462
['http://crl3.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crl', 'http://crl4.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crl'] 11 8.461538461538462
------------------------------------------------------
OCSP Server：
['http://r3.o.lencr.org'] 489 61.820480404551205
['http://ocsp.digicert.com'] 63 7.964601769911504
['http://zerossl.ocsp.sectigo.com'] 56 7.079646017699115
['http://ocsp.sectigo.com'] 38 4.804045512010114
['http://statuse.digitalcertvalidation.com'] 24 3.034134007585335
['http://ocsp.usertrust.com'] 16 2.0227560050568902
['http://e1.o.lencr.org'] 16 2.0227560050568902
['http://ocsp.int-x3.letsencrypt.org'] 14 1.7699115

## 分析撤销响应 

### 整体分析 

In [86]:
all_raw.keys()

Index(['domain', 'ip', 'tlsVersion', 'scsv', 'connError', 'raw_cert_chain',
       'cert_level', 'certValid', 'caName', 'time_valid', 'cert_lifetime',
       'cert_error', 'tlsHave', 'sct_tls_valid', 'tls_log', 'certHave',
       'cert_valid', 'cert_log', 'ocspHave', 'ocsp_valid', 'ocsp_log',
       'flagValid', 'crLs', 'crl_server', 'ocsp', 'ocspServer', 'ocspStapling',
       'ocspMustStaple', 'respect_ms', 'response_flag', 'crl_cert_status',
       'crl_response_sig', 'ocsp_cert_status', 'ocsp_response_sig',
       'ocsp_stapling_cert_status', 'ocsp_stapling_response_sig', 'caa_result',
       'tlsa_dnssec', 'caa_dnssec', 'dnssec', 'dane', 'dane_err',
       'tls13_down_12', 'tls13_down_11', 'tls13_down_10', 'http_code',
       'http_headers', 'country', 'isp', 'org', 'tlsa_flag', 'caa_flag'],
      dtype='object')

In [87]:
print("在所有DOT中，可以获取到撤销响应有：")
print(all_raw["response_flag"].value_counts())
print("-------------------------------")
print(all_raw["response_flag"].value_counts()/all_num*100)


在所有DOT中，可以获取到撤销响应有：
True     693
False    204
Name: response_flag, dtype: int64
-------------------------------
True     77.257525
False    22.742475
Name: response_flag, dtype: float64


In [88]:
print("在支持证书撤销的DOT中，可以获取到撤销响应有：")
print(have_revoke["response_flag"].value_counts())
print("-------------------------------")
print(have_revoke["response_flag"].value_counts()/len(have_revoke)*100)

在支持证书撤销的DOT中，可以获取到撤销响应有：
True     693
False    100
Name: response_flag, dtype: int64
-------------------------------
True     87.38966
False    12.61034
Name: response_flag, dtype: float64


### 分析证书状态 

#### 查看有多少证书被撤销 

In [89]:
revoked = all_raw[(all_raw["crl_cert_status"]=="Revoked")|(all_raw["ocsp_cert_status"]=="Revoked")|(all_raw["ocsp_stapling_cert_status"]=="Revoked")]
revoked

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


In [90]:
all_raw["crl_cert_status"].value_counts()
all_raw["ocsp_cert_status"].value_counts()
all_raw["ocsp_stapling_cert_status"].value_counts()

         767
Good     123
Error      7
Name: crl_cert_status, dtype: int64

Good     675
Error    116
         106
Name: ocsp_cert_status, dtype: int64

        695
Good    202
Name: ocsp_stapling_cert_status, dtype: int64

#### 分析不同撤销机制的差异 

In [91]:
print("CRL和OCSP差异:")
revoked[((revoked["crl_cert_status"]=="Good")&(revoked["ocsp_cert_status"]=="Revoked"))|((revoked["crl_cert_status"]=="Revoked")&(revoked["ocsp_cert_status"]=="Good"))]
print("CRL和OCSPstapling差异:")
revoked[((revoked["crl_cert_status"]=="Good")&(revoked["ocsp_stapling_cert_status"]=="Revoked"))|((revoked["crl_cert_status"]=="Revoked")&(revoked["ocsp_stapling_cert_status"]=="Good"))]
print("OCSPstapling和OCSP差异:")
revoked[((revoked["ocsp_stapling_cert_status"]=="Good")&(revoked["ocsp_cert_status"]=="Revoked"))|((revoked["ocsp_stapling_cert_status"]=="Revoked")&(revoked["ocsp_cert_status"]=="Good"))]

CRL和OCSP差异:


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


CRL和OCSPstapling差异:


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


OCSPstapling和OCSP差异:


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag


## 分析撤销响应的签名 

In [92]:
print("CRL响应签名：")
print(have_crl["crl_response_sig"].value_counts())
print("-------------------------------")
print(have_crl["crl_response_sig"].value_counts()/len(have_crl)*100)
print("**************************************")
print("OCSP响应签名")
print(have_ocsp["ocsp_response_sig"].value_counts())
print("-------------------------------")
print(have_ocsp["ocsp_response_sig"].value_counts()/len(have_ocsp)*100)
print("**************************************")
print("OCSPStapling响应签名")
print(have_ocspsl["ocsp_stapling_response_sig"].value_counts())
print("-------------------------------")
print(have_ocspsl["ocsp_stapling_response_sig"].value_counts()/len(have_ocspsl)*100)

CRL响应签名：
True     124
False      6
Name: crl_response_sig, dtype: int64
-------------------------------
True     95.384615
False     4.615385
Name: crl_response_sig, dtype: float64
**************************************
OCSP响应签名
True     637
False    154
Name: ocsp_response_sig, dtype: int64
-------------------------------
True     80.530973
False    19.469027
Name: ocsp_response_sig, dtype: float64
**************************************
OCSPStapling响应签名
True     200
False      2
Name: ocsp_stapling_response_sig, dtype: int64
-------------------------------
True     99.009901
False     0.990099
Name: ocsp_stapling_response_sig, dtype: float64


### 分析签名的CA 

In [93]:
print("CRL响应签名正确的CA")
for index,value in have_crl[have_crl["crl_response_sig"]==True].caName.value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_crl)*100) 
        
print("**************************************")

print("CRL响应签名错误的CA")
for index,value in have_crl[have_crl["crl_response_sig"]==False].caName.value_counts().iteritems():
    print(index,value,value/len(have_crl)*100)

print("**************************************")

print("OCSP响应签名正确的CA")
for index,value in have_ocsp[have_ocsp["crl_response_sig"]==True].caName.value_counts().iteritems():
    if value >= 10:
        print(index,value,value/len(have_ocsp)*100) 
        
print("**************************************")

print("OCSP响应签名错误的CA")
for index,value in have_ocsp[have_ocsp["crl_response_sig"]==False].caName.value_counts().iteritems():
    print(index,value,value/len(have_ocsp)*100)

CRL响应签名正确的CA
['DigiCert Inc'] 46 35.38461538461539
['GoGetSSL'] 14 10.76923076923077
['Cloudflare, Inc.'] 11 8.461538461538462
**************************************
CRL响应签名错误的CA
['Chunghwa Telecom Co., Ltd.'] 2 1.5384615384615385
['CloudFlare, Inc.'] 2 1.5384615384615385
['GeoTrust Inc.'] 1 0.7692307692307693
['British Telecommunications plc'] 1 0.7692307692307693
**************************************
OCSP响应签名正确的CA
['DigiCert Inc'] 46 5.815423514538559
['GoGetSSL'] 14 1.7699115044247788
['Cloudflare, Inc.'] 11 1.390644753476612
**************************************
OCSP响应签名错误的CA
["Let's Encrypt"] 519 65.61314791403287
['ZeroSSL'] 56 7.079646017699115
['Sectigo Limited'] 38 4.804045512010114
['TrustAsia Technologies, Inc.'] 28 3.5398230088495577
['DigiCert Inc'] 13 1.643489254108723
['GlobalSign nv-sa'] 4 0.5056890012642226
['TrustOcean Limited'] 3 0.37926675094816686
['Chunghwa Telecom Co., Ltd.'] 2 0.2528445006321113
['CentralNic Luxembourg Sàrl'] 2 0.2528445006321113
['GeoTrust In

# 对DANE-TLSA进行分析 

## DNSSEC

In [40]:
dnssec_raw = pd.read_csv("./edns-dnssec/doh_edns_success.txt")
dnssec_raw = pd.DataFrame(dnssec_raw,columns = ['ip','dnssec'])

dnssec_raw

,ip,dnssec
0,1.0.0.2,True
1,130.61.56.82,True
2,168.138.51.151,True
3,37.221.215.77,True
4,1.0.0.1,True
...,...,...
13486,52.42.134.63,False
13487,63.240.152.245,False
13488,63.240.152.245,False
13489,63.240.152.245,False


In [41]:
all_raw = pd.merge(all_raw,dnssec_raw,on=["ip"],how="left")
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag,dnssec_y
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,True,True,200.0,Strict-Transport-Security: max-age=31536000; i...,Australia,"Cloudflare, Inc",APNIC and Cloudflare DNS Resolver project,False,False,True
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,True,True,404.0,none,Japan,Oracle Corporation,Oracle Cloud Infrastructure (ap-tokyo-1),False,False,False
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,False,False,404.0,none,Canada,Windscribe,Zenu Software LLC,False,False,NaN
3,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,Germany,"nextdns, Inc.","NextDNS, Inc",False,False,True
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,False,False,405.0,Strict-Transport-Security: max-age=63072000; i...,Germany,"nextdns, Inc.","NextDNS, Inc",False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,[Let's Encrypt],True,...,False,False,404.0,none,Poland,Aruba S.p.A.,Aruba S.p.A. - Cloud Services,False,False,NaN
1085,us02.dns4me.net,54.172.110.180,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdiRE...,DV,True,[ZeroSSL],True,...,True,True,400.0,none,United States,"Amazon.com, Inc.",AWS EC2 (us-east-1),False,False,True
1086,us02.dns4me.net,54.172.110.180,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdiRE...,DV,True,[ZeroSSL],True,...,True,True,400.0,none,United States,"Amazon.com, Inc.",AWS EC2 (us-east-1),False,False,True
1087,us02.dns4me.net,54.172.110.180,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdiRE...,DV,True,[ZeroSSL],True,...,True,True,400.0,none,United States,"Amazon.com, Inc.",AWS EC2 (us-east-1),False,False,True


In [43]:
have_dnssec = all_raw[all_raw["dnssec_y"]==True]
print("在所有DOT中，拥有受DNSSEC保护的TLSA记录的有：{}, 占比{:.2%}".format(len(have_dnssec), len(have_dnssec)/all_num))


在所有DOT中，拥有受DNSSEC保护的TLSA记录的有：658, 占比74.18%


## 分析DOT的TLSA

In [38]:
have_tlsa = all_raw[all_raw["tlsa_flag"]==True]
print("在所有DOT中，拥有TLSA记录的有：{}, 占比{:.2%}".format(len(have_tlsa), len(have_tlsa)/all_num))
tlsa_dnssec = all_raw[(all_raw["tlsa_flag"]==True)&(all_raw["tlsa_dnssec"]=="Secure")]
print("在所有DOT中，拥有受DNSSEC保护的TLSA记录的有：{}, 占比{:.2%}".format(len(tlsa_dnssec), len(tlsa_dnssec)/all_num))
print("在有TLSA记录的DOT中，拥有受DNSSEC保护的TLSA记录的有：{}, 占比{:.2%}".format(len(tlsa_dnssec), len(tlsa_dnssec)/len(have_tlsa)))
print("受DNSSEC保护的TLSA")
tlsa_dnssec

在所有DOT中，拥有TLSA记录的有：31, 占比3.49%
在所有DOT中，拥有受DNSSEC保护的TLSA记录的有：30, 占比3.38%
在有TLSA记录的DOT中，拥有受DNSSEC保护的TLSA记录的有：30, 占比96.77%
受DNSSEC保护的TLSA


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
354,anycast.censurfridns.nu,198.180.150.12,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZjak...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,United States,RGnet OU,RGnet OU,True,False
5021,anycast.uncensoreddns.org,130.225.244.166,,False,dial tcp 130.225.244.166:443: connect: connect...,,,False,none,False,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
6993,deic-ore.anycast.uncensoreddns.org,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
47507,anycast.censurfridns.dk,130.225.244.166,,False,dial tcp 130.225.244.166:443: connect: connect...,,,False,none,False,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
52383,unicast.censurfridns.nu,89.233.43.71,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUUxRE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Telia Company AB,WEBPARTNER,True,False
68452,deic-lgb.anycast.censurfridns.nu,130.225.244.166,,False,dial tcp 130.225.244.166:443: connect: connect...,,,False,none,False,...,False,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
68459,doh.bortzmeyer.fr,193.70.85.11,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,France,OVH SAS,OVH,True,False
84308,unicast.uncensoreddns.org,89.233.43.71,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUUxRE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Telia Company AB,WEBPARTNER,True,False
84309,deic-ore.anycast.censurfridns.dk,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
84804,anycast.uncensoreddns.dk,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False


### 分析受DNSSEC保护的TLSA的域名org和ISP 

In [95]:
print("org:")
tlsa_dnssec["org"].value_counts()
print("************************")
print("ISP:")
tlsa_dnssec["isp"].value_counts()

org:


Danmarks Tekniske Universitet    11
WEBPARTNER                        4
Kracon ApS                        4
Thomas Steen Rasmussen            3
RGnet OU                          2
OVH                               1
none                              1
netcup GmbH                       1
Scholz-Kallies Rene               1
                                  1
Fasthosts Internet Limited        1
Name: org, dtype: int64

************************
ISP:


Forskningsnettet                          11
Telia Company AB                           4
Kracon ApS                                 4
UNCENSOREDDNS                              3
RGnet OU                                   2
OVH SAS                                    1
none                                       1
netcup GmbH                                1
OVH ISP                                    1
GANDI is an ICANN accredited registrar     1
IONOS SE                                   1
Name: isp, dtype: int64

## 验证DOT的DANE 

In [39]:
have_dane = all_raw[all_raw["dane"]==True]
print("在所有DOT中，支持DANE的有：{}, 占比{:.2%}".format(len(have_dane), len(have_dane)/all_num))
print("在有受DNSSEC保护的TLSA记录的DOT中，支持DANE的有：{}, 占比{:.2%}".format(len(have_dane), len(have_dane)/len(tlsa_dnssec)))
print("支持DANE的DOT")
have_dane

在所有DOT中，支持DANE的有：29, 占比3.27%
在有受DNSSEC保护的TLSA记录的DOT中，支持DANE的有：29, 占比96.67%
支持DANE的DOT


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
354,anycast.censurfridns.nu,198.180.150.12,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZjak...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,United States,RGnet OU,RGnet OU,True,False
5021,anycast.uncensoreddns.org,130.225.244.166,,False,dial tcp 130.225.244.166:443: connect: connect...,,,False,none,False,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
6993,deic-ore.anycast.uncensoreddns.org,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
47507,anycast.censurfridns.dk,130.225.244.166,,False,dial tcp 130.225.244.166:443: connect: connect...,,,False,none,False,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
52383,unicast.censurfridns.nu,89.233.43.71,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUUxRE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Telia Company AB,WEBPARTNER,True,False
68452,deic-lgb.anycast.censurfridns.nu,130.225.244.166,,False,dial tcp 130.225.244.166:443: connect: connect...,,,False,none,False,...,False,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
68459,doh.bortzmeyer.fr,193.70.85.11,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,France,OVH SAS,OVH,True,False
84308,unicast.uncensoreddns.org,89.233.43.71,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUUxRE...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Telia Company AB,WEBPARTNER,True,False
84309,deic-ore.anycast.censurfridns.dk,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False
84804,anycast.uncensoreddns.dk,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,True,False,False,400.0,none,Denmark,Forskningsnettet,Danmarks Tekniske Universitet,True,False


### 分析DANE验证错误 

In [97]:
print(have_tlsa["dane_err"].value_counts())
print("-------------------------------")
print(have_tlsa["dane_err"].value_counts()/len(have_tlsa)*100)

no_err       29
insecure      1
match_err     1
Name: dane_err, dtype: int64
-------------------------------
no_err       93.548387
insecure      3.225806
match_err     3.225806
Name: dane_err, dtype: float64


In [98]:
have_tlsa[have_tlsa["dane_err"]=="insecure"]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_12,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag
528,public.dns.iij.jp,103.2.57.5,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjak...,OV,True,['GlobalSign nv-sa'],True,...,True,False,False,none,none,Japan,Internet Initiative Japan Inc.,Internet Initiative Japan Inc.,True,False


### 分析支持DANE的org和ISP 

In [99]:
print("org:")
have_dane["org"].value_counts()
print("************************")
print("ISP:")
have_dane["isp"].value_counts()

org:


Danmarks Tekniske Universitet    11
WEBPARTNER                        4
Kracon ApS                        4
Thomas Steen Rasmussen            3
RGnet OU                          2
OVH                               1
none                              1
netcup GmbH                       1
Scholz-Kallies Rene               1
Fasthosts Internet Limited        1
Name: org, dtype: int64

************************
ISP:


Forskningsnettet    11
Telia Company AB     4
Kracon ApS           4
UNCENSOREDDNS        3
RGnet OU             2
OVH SAS              1
none                 1
netcup GmbH          1
OVH ISP              1
IONOS SE             1
Name: isp, dtype: int64

# 分析DOT的CAA 

In [100]:
have_caa = all_raw[all_raw["caa_flag"]==True]
print("在所有DOT中，拥有CAA记录的有：{}, 占比{:.2%}".format(len(have_caa), len(have_caa)/all_num))

在所有DOT中，拥有CAA记录的有：24, 占比2.68%


In [101]:
import difflib
def string_similar(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio() ## 比较两个字符串的相似比例

In [102]:
must_caa=0
match_caa=0
must_match_caa=0
match_ca = []
match_org = []

for i in range(len(have_caa)):
    if have_caa.iloc[i]["caName"] != "none":
        caa_list=have_caa.iloc[i]["caa_result"].split(";")
        caname = ''.join(have_caa.iloc[i]["caName"]) 
        for caa_data in caa_list:
            if len(caa_data.split(" "))==3:
                caa_flag = caa_data.split(" ")[0]
                caa_type = caa_data.split(" ")[1]
                caa_value = caa_data.split(" ")[2]
                if caa_flag == "128":
                    must_caa += 1
                if string_similar(caa_value.lower(), caname) > 0.5:
                    match_caa += 1
                    match_ca.append(caname)
                    match_org.append(have_caa.iloc[i]["org"])
                    if caa_flag == "128":
                        must_match_caa += 1
                    break                
                    

print("在拥有CAA记录的DOT中，CAA记录匹配的有：{}, 占比{:.2%}".format(match_caa, match_caa/len(have_caa)))
print("在拥有CAA记录的DOT中，CAA记录的flag为128的有：{}, 占比{:.2%}".format(must_caa, must_caa/len(have_caa)))
print("在CAA记录的flag为128的DOT中，CAA记录匹配的有：{}, 占比{:.2%}".format(must_match_caa , must_match_caa /must_caa))
match_ca
match_org

在拥有CAA记录的DOT中，CAA记录匹配的有：11, 占比45.83%
在拥有CAA记录的DOT中，CAA记录的flag为128的有：4, 占比16.67%
在CAA记录的flag为128的DOT中，CAA记录匹配的有：2, 占比50.00%


['["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]',
 '["Let\'s Encrypt"]']

['Wikimedia Foundation Inc',
 'Dolphin IT-Systeme e.K.',
 'none',
 'Hathway Cable and Datacom Limited',
 'none',
 'Microsoft Azure Cloud (uksouth)',
 'netcup GmbH',
 'Dolphin IT-Systeme e.K.',
 'Dolphin IT-Systeme e.K.',
 'none',
 'OVH SAS']

# 分析HTTP头

## 处理HTTP头源数据

In [19]:
def parse_http_headers(v):
    input = str(v)
    d = dict()
    for line in input.splitlines(): # Python splitlines() 按照行('\r', '\r\n', \n')分隔，返回一个包含各行作为元素的列表
        a = line.split(':')
        if a[0] == "Strict-Transport-Security":
            if len(a) != 2:
                parse_errors.append("sts too many elements: {}".format(line))
            else:
                for b in a[1].split(";"):
                    b = b.strip()
                    maxage_search = re.search('max-age=(.*)', b)
                    if maxage_search:
                        d["hsts"] = True
                        d["hsts_maxage"] = maxage_search.group(1)
                    elif b.lower() == "includesubdomains":
                        d["hsts_include_sub"] = True
                    elif b == "preload":
                        d["hsts_preload"] = True
                    elif b == "requireCT":
                        d["hsts_requireCT"] = True
                    elif b == "":
                        continue
                    else:
                        parse_errors.append("unknown sts header: -{}- in line -{}-".format(b,line))
                        continue
            continue
        elif a[0] == "Public-Key-Pins" or a[0] == "Public-Key-Pins-Report-Only":
            if a[0] == "Public-Key-Pins":
                PREFIX = "http-header--hpkp"
            else:
                PREFIX = "http-header--hpkp-ro"
            d[PREFIX] = True
            val = ':'.join(a[1:]) ## hpkp has : in eg report-uri, so re-join the previously split a
            for b in val.split(";"):
                b = b.strip()
                pinsha256_search = re.search('pin-sha256=(.*)', b)
                maxage_search = re.search('max-age=(.*)', b)
                reporturi_search = re.search('report-uri=(.*)', b)
                if pinsha256_search:
                    if PREFIX + "-pinsha256" in d:
                        d[PREFIX + "-pinsha256"].append(pinsha256_search.group(1))
                    else:
                        d[PREFIX + "-pinsha256"] = [pinsha256_search.group(1)]
                elif maxage_search:
                    d[PREFIX + "-max-age"] = maxage_search.group(1)
                elif reporturi_search:
                    d[PREFIX + "-report-uri"] = reporturi_search.group(1)
                elif b.lower() == "includesubdomains":
                    d[PREFIX + "-includeSubDomains"] = True
                elif b == "":
                    continue
                else:
                    parse_errors.append("unknown hpkp header: -{}- in line -{}-".format(b, line))
                    d.clear()
                    continue
        elif a[0] == "Expect-CT":
            d["expect_ct"] = True
            ct_val = ':'.join(a[1:]) ## expect-ct has report-uri too
            if ct_val.find(",") > 0:
                for b in ct_val.split(","):
                    b = b.strip()
                    maxage_search = re.search('max-age=(.*)', b)
                    reporturi_search = re.search('report-uri=(.*)', b)
                    if maxage_search:
                        d["expect_ct_maxage"] = maxage_search.group(1)
                    elif reporturi_search:
                        d["expect_ct_report_uri"] = reporturi_search.group(1)
                    elif b == "enforce":
                        d["expect_ct_enforce"] = True
                    elif b.lower() == "preload":
                        d["expect_ct_preload"] = True
                    elif b == "":
                        continue
                    else:
                        parse_errors.append("unknown expect-ct header: -{}- in line -{}-".format(b, line))
                        d.clear()
                        continue
            else:
                for b in ct_val.split(";"):
                    b = b.strip()
                    maxage_search = re.search('max-age=(.*)', b)
                    reporturi_search = re.search('report-uri=(.*)', b)
                    if maxage_search:
                        d["expect_ct_maxage"] = maxage_search.group(1)
                    elif reporturi_search:
                        d["expect_ct_report_uri"] = reporturi_search.group(1)
                    elif b == "enforce":
                        d["expect_ct_enforce"] = True
                    elif b.lower() == "preload":
                        d["expect_ct_preload"] = True
                    elif b == "":
                        continue
                    else:
                        parse_errors.append("unknown expect-ct header: -{}- in line -{}-".format(b, line))
                        d.clear()
                        continue
        else:
            parse_errors.append("unknown header: -{}-".format(line))
            d.clear()
            continue
    
    if len(d) > 0:
        for i in d.keys():
            global colums_to_create
            columns_to_create[i] = 1
        return d
    else:
        return

In [20]:
def create_colums(a_raw_http):
    d = a_raw_http.assign(httpdict=a_raw_http.http_headers.apply(lambda x: parse_http_headers(x))) ## 添加httpdict
    d_validly_parsed = d.dropna(subset=["httpdict"]) ## 滤除缺失值的行，指定为httpdict列
    print("Parsed headers for {} columns, {} left after dropping nans.".format(d.domain.count(), d_validly_parsed.domain.count()))
    for i in columns_to_create.keys(): ## 根据输出key新建列
        print(i)
        d_validly_parsed[i] = np.NaN ## 新建列，存储httpheader
    return d_validly_parsed

In [21]:
columns_to_create = dict()
parse_errors = [] 
da_raw_http = create_colums(all_raw)

Parsed headers for 725 columns, 114 left after dropping nans.
hsts
hsts_maxage
hsts_include_sub
hsts_preload
expect_ct
expect_ct_maxage
expect_ct_report_uri
expect_ct_enforce


C:\Users\11624\AppData\Local\Temp/ipykernel_2152/1311509257.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_validly_parsed[i] = np.NaN ## 新建列，存储httpheader


In [22]:
def dict2column(dfin):
    for x in dfin["httpdict"].keys():
        dfin[x] = dfin["httpdict"][x]
    return dfin

In [23]:
da_raw_http_in = da_raw_http.apply(lambda x: dict2column(x), axis=1)

In [24]:
da_raw_http_in = pd.DataFrame(da_raw_http_in,columns = ['domain','ip','httpdict','hsts','hsts_maxage','hsts_include_sub','hsts_preload','expect_ct','expect_ct_maxage','expect_ct_report_uri','expect_ct_enforce'])
da_raw_http_in

,domain,ip,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce
0,cloudflare-dns.com,1.0.0.1,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
4,dns.nextdns.io,185.253.5.128,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
5,dns.sb,185.37.252.149,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
10,lax-1.edge.nextdns.io,23.247.214.45,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
12,dal-1.edge.nextdns.io,23.247.215.95,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
134474,doh.j2b.fr,5.135.155.46,"{'hsts': True, 'hsts_maxage': '31536000'}",True,31536000,NaN,NaN,NaN,NaN,NaN,NaN
134545,dns.nauval.my.id,103.214.113.104,"{'hsts': True, 'hsts_maxage': '31536000'}",True,31536000,NaN,NaN,NaN,NaN,NaN,NaN
134566,doh.li,46.101.66.244,"{'hsts': True, 'hsts_maxage': '31536000'}",True,31536000,NaN,NaN,NaN,NaN,NaN,NaN
134610,tyo-3.edge.nextdns.io,146.19.14.10,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN


In [25]:
all_raw = pd.merge(all_raw,da_raw_http_in,how="left")
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,caa_flag,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,False,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,False,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
4,dns.sb,185.37.252.149,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdIek...,DV,True,[GoGetSSL],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,dns.seey6.xyz,60.205.210.33,,False,read tcp 172.22.21.0:57162->60.205.210.33:443:...,,,False,none,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,dns.nguyendn.com,103.237.147.61,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjVE...,DV,True,[Let's Encrypt],True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
722,doh.dnsfilter.com,103.247.36.150,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,[Let's Encrypt],True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,[Let's Encrypt],True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
all_raw.to_csv("../doh-domain-tlscert-uniq-1036.csv",index=None)

## 分析HSTS 

In [31]:
hsts = all_raw[all_raw["hsts"]==True]
print('在{}个DOH中，受HSTS保护的有{}，占比{:.2%}'.format(all_num,len(hsts),len(hsts)/all_num))


在887个DOH中，受HSTS保护的有142，占比16.01%


### 分析HSTS的Max-Age 

In [52]:
for index,value in hsts["hsts_maxage"].value_counts().iteritems():
    print('maxage: {}days, 数量: {},在有hsts的DOT中占比{:.2%},在所有DOT中占比{:.2%}'.format(int(index)/(3600*24),value, value/len(hsts),value/all_num))

maxage: 730.0days, 数量: 319,在有hsts的DOT中占比80.15%,在所有DOT中占比35.96%
maxage: 365.0days, 数量: 64,在有hsts的DOT中占比16.08%,在所有DOT中占比7.22%
maxage: 182.5days, 数量: 7,在有hsts的DOT中占比1.76%,在所有DOT中占比0.79%
maxage: 190.0days, 数量: 4,在有hsts的DOT中占比1.01%,在所有DOT中占比0.45%
maxage: 180.0days, 数量: 2,在有hsts的DOT中占比0.50%,在所有DOT中占比0.23%
maxage: 1231.3045138888888days, 数量: 1,在有hsts的DOT中占比0.25%,在所有DOT中占比0.11%
maxage: 356.0days, 数量: 1,在有hsts的DOT中占比0.25%,在所有DOT中占比0.11%


In [53]:
hsts[hsts["hsts_maxage"].isnull()]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,dnssec_y,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce


### 分析HSTS的incldesubdomains

In [54]:
hsts

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,dnssec_y,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,True,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
3,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
4,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
5,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
6,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,tyo-3.edge.nextdns.io,146.19.14.10,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,[Let's Encrypt],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
1275,tyo-3.edge.nextdns.io,146.19.14.10,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,[Let's Encrypt],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
1276,tyo-3.edge.nextdns.io,146.19.14.10,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR6ek...,DV,True,[Let's Encrypt],True,...,True,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN
1279,dns.vorlif.org,85.235.65.70,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVvek...,DV,True,[Let's Encrypt],True,...,True,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,NaN,NaN,NaN,NaN


In [61]:
hsts["hsts_include_sub"].value_counts()
hsts["hsts_include_sub"].value_counts()/len(hsts)*100

True    283
Name: hsts_include_sub, dtype: int64

True    71.105528
Name: hsts_include_sub, dtype: float64

### 分析HSTS的perload 

In [57]:
hsts["hsts_preload"].value_counts()
hsts["hsts_preload"].value_counts()/len(hsts)*100

True    260
Name: hsts_preload, dtype: int64

True    65.326633
Name: hsts_preload, dtype: float64

#### 分析有多少设置了HSTS perload的域名包含在HSTS_staic_json中 

In [58]:
# https://github.com/chromium/chromium/blob/master/net/http/transport_security_state_static.json
# 2021-11-10

preload_file = "../hsts_perload_list/transport_security_state_static.json"
import json
with open(preload_file) as plfile:
    pl = json.load(plfile)

In [59]:
hstspldomainsd = dict()
for i in pl["entries"]:
    if "force-https" in i.values():
        hstspldomainsd[i["name"]] = 1
        
hstspldomain = set(hstspldomainsd.keys())
len(hstspldomain)

156924

In [60]:
hsts_pre = all_raw[all_raw["hsts_preload"] == True]
uniq_hsts_preload_domains = set(hsts_pre.domain)
intersect = hstspldomain & uniq_hsts_preload_domains

print('一共有{}个域名设置了HSTS_perload且出现在公开json文件中，在有hsts的DOH中占比{:.2%},在有hsts_perload的DOH中占比{:.2%}'.format(len(intersect),len(intersect)/len(hsts), len(intersect)/len(hsts_pre)))

一共有5个域名设置了HSTS_perload且出现在公开json文件中，在有hsts的DOH中占比1.26%,在有hsts_perload的DOH中占比1.92%


## 分析Expect-CT 

In [120]:
expet_ct = all_raw[all_raw["expect_ct"]==True]
print('在{}个DOH中，受Expect-CT保护的有{}，占比{:.2%}'.format(all_num,len(expet_ct),len(expet_ct)/all_num))
expet_ct

在897个DOH中，受Expect-CT保护的有5，占比0.56%


,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,caa_flag,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,['DigiCert Inc'],True,...,False,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
49,doh.intouchstaging.com,170.176.148.150,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUcyek...,OV,True,['DigiCert Inc'],True,...,False,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
222,family.cloudflare-dns.com,1.1.1.3,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZrVE...,OV,True,['DigiCert Inc'],True,...,False,"{'expect_ct': True, 'expect_ct_maxage': '60480...",NaN,NaN,NaN,NaN,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
456,security.cloudflare-dns.com,1.1.1.2,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZsVE...,OV,True,['DigiCert Inc'],True,...,False,"{'expect_ct': True, 'expect_ct_maxage': '60480...",NaN,NaN,NaN,NaN,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN
796,doh.zakaria.website,213.171.210.111,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVkRE...,DV,True,"[""Let's Encrypt""]",True,...,False,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,86400,NaN,True


### 分析Expect-CT的Max-Age 

In [121]:
for index,value in expet_ct["expect_ct_maxage"].value_counts().iteritems():
    print('maxage: {}days, 数量: {},在有Expect-CT的DOT中占比{:.2%},在所有DOT中占比{:.2%}'.format(int(index)/(3600*24),value, value/len(expet_ct),value/all_num))

maxage: 7.0days, 数量: 4,在有Expect-CT的DOT中占比80.00%,在所有DOT中占比0.45%
maxage: 1.0days, 数量: 1,在有Expect-CT的DOT中占比20.00%,在所有DOT中占比0.11%


In [122]:
expet_ct[expet_ct["expect_ct_maxage"].isnull()]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,caa_flag,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce


### 分析Expect-CT的enforce 

In [123]:
expet_ct["expect_ct_enforce"].value_counts()
expet_ct["expect_ct_enforce"].value_counts()/len(expet_ct)
# print('有{}个域名设置了Expect-CT enforce, 在有Expect-CT的域名中占比{:.2%},在所有DOH中占比{:.2%}'.format())

True    1
Name: expect_ct_enforce, dtype: int64

True    0.2
Name: expect_ct_enforce, dtype: float64

In [124]:
expet_ct[expet_ct["expect_ct_enforce"]==True]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,caa_flag,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce
796,doh.zakaria.website,213.171.210.111,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVkRE...,DV,True,"[""Let's Encrypt""]",True,...,False,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,86400,NaN,True


####  没有Expect-CT preload

### 在有Expect-CT时是否提供有效SCT 

In [125]:
expect_valid_sct = all_raw[(all_raw["expect_ct"]==True) & (all_raw["flagValid"]==True)]
print("有{}个DOH 支持Expect-CT且提供了有效的sct，在提供Expect-CT的DOH中占比{:.2%}".format(len(expect_valid_sct),len(expect_valid_sct)/len(expet_ct)))

有5个DOH 支持Expect-CT且提供了有效的sct，在提供Expect-CT的DOH中占比100.00%


In [126]:
expect_invalid_sct = all_raw[(all_raw["expect_ct"]==True) & (all_raw["flagValid"]==False)]
expect_invalid_sct
expect_invalid_sct["cert_error"]

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,caa_flag,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce


Series([], Name: cert_error, dtype: object)

# 分析DOT的公司 

In [127]:
all_raw.org = all_raw.org.apply(lambda x: x.lower())

In [128]:
for index,value in all_raw["org"].value_counts().iteritems():
    if value >= 20:
        print(index,value,value/len(all_raw)*100) 

 63 7.023411371237458
digitalocean, llc 33 3.678929765886288
none 30 3.3444816053511706
hetzner 28 3.121516164994426


In [129]:
digitalocean = all_raw[all_raw["org"].str.contains("digitalocean")]
print("digitalocean",len(digitalocean), len(digitalocean)/all_num*100)
apple  = all_raw[all_raw["org"].str.contains("apple")]
print("apple",len(apple), len(apple)/all_num*100)
virtual = all_raw[all_raw["org"].str.contains("virtual")]
print("virtual",len(virtual), len(virtual)/all_num*100)
hetzner = all_raw[all_raw["org"].str.contains("hetzner")]
print("hetzner",len(hetzner), len(hetzner)/all_num*100)

digitalocean 34 3.79041248606466
apple 5 0.5574136008918618
virtual 14 1.560758082497213
hetzner 34 3.79041248606466


In [130]:
all_raw.keys()

Index(['domain', 'ip', 'tlsVersion', 'scsv', 'connError', 'raw_cert_chain',
       'cert_level', 'certValid', 'caName', 'time_valid', 'cert_lifetime',
       'cert_error', 'tlsHave', 'sct_tls_valid', 'tls_log', 'certHave',
       'cert_valid', 'cert_log', 'ocspHave', 'ocsp_valid', 'ocsp_log',
       'flagValid', 'crLs', 'crl_server', 'ocsp', 'ocspServer', 'ocspStapling',
       'ocspMustStaple', 'respect_ms', 'response_flag', 'crl_cert_status',
       'crl_response_sig', 'ocsp_cert_status', 'ocsp_response_sig',
       'ocsp_stapling_cert_status', 'ocsp_stapling_response_sig', 'caa_result',
       'tlsa_dnssec', 'caa_dnssec', 'dnssec', 'dane', 'dane_err',
       'tls13_down_12', 'tls13_down_11', 'tls13_down_10', 'http_code',
       'http_headers', 'country', 'isp', 'org', 'tlsa_flag', 'caa_flag',
       'httpdict', 'hsts', 'hsts_maxage', 'hsts_include_sub', 'hsts_preload',
       'expect_ct', 'expect_ct_maxage', 'expect_ct_report_uri',
       'expect_ct_enforce'],
      dtype='object')

## 分析TLS版本 

In [131]:
print("digitalocean")
for index,value in digitalocean["tlsVersion"].value_counts().iteritems():
    print(index,value,"---",value/len(digitalocean)*100) 
print("*******************************************")
print("apple")
for index,value in apple["tlsVersion"].value_counts().iteritems():
    print(index,value,"---",value/len(apple)*100) 
print("*******************************************")
print("virtual")
for index,value in virtual["tlsVersion"].value_counts().iteritems():
    print(index,value,"---",value/len(virtual)*100) 
print("*******************************************")
print("hetzner")
for index,value in hetzner["tlsVersion"].value_counts().iteritems():
    print(index,value,"---",value/len(hetzner)*100) 

digitalocean
TLSv1.3 27 --- 79.41176470588235
TLSv1.2 5 --- 14.705882352941178
 2 --- 5.88235294117647
*******************************************
apple
 4 --- 80.0
TLSv1.3 1 --- 20.0
*******************************************
virtual
TLSv1.3 9 --- 64.28571428571429
 4 --- 28.57142857142857
TLSv1.2 1 --- 7.142857142857142
*******************************************
hetzner
TLSv1.3 30 --- 88.23529411764706
TLSv1.2 4 --- 11.76470588235294


## 分析TLS降级保护机制 

In [132]:
digitalocean_scsv_num = digitalocean.query('(tlsVersion == "TLSv1.2" & scsv == True)| (tlsVersion == "TLSv1.3" & tls13_down_12 == True)').ip.count()
print('{}个DOT属于digitalocean，其中受TLS降级保护的有{}，占比{:.2%}'.format(len(digitalocean),digitalocean_scsv_num, digitalocean_scsv_num/len(digitalocean)))

apple_scsv_num = apple.query('(tlsVersion == "TLSv1.2" & scsv == True)| (tlsVersion == "TLSv1.3" & tls13_down_12 == True)').ip.count()
print('{}个DOT属于apple，其中受TLS降级保护的有{}，占比{:.2%}'.format(len(apple),apple_scsv_num, apple_scsv_num/len(apple)))

virtual_scsv_num = virtual.query('(tlsVersion == "TLSv1.2" & scsv == True)| (tlsVersion == "TLSv1.3" & tls13_down_12 == True)').ip.count()
print('{}个DOT属于virtual，其中受TLS降级保护的有{}，占比{:.2%}'.format(len(virtual),virtual_scsv_num, virtual_scsv_num/len(virtual)))

hetzner_scsv_num = hetzner.query('(tlsVersion == "TLSv1.2" & scsv == True)| (tlsVersion == "TLSv1.3" & tls13_down_12 == True)').ip.count()
print('{}个DOT属于hetzner，其中受TLS降级保护的有{}，占比{:.2%}'.format(len(hetzner),hetzner_scsv_num, hetzner_scsv_num/len(hetzner)))

34个DOT属于digitalocean，其中受TLS降级保护的有27，占比79.41%
5个DOT属于apple，其中受TLS降级保护的有1，占比20.00%
14个DOT属于virtual，其中受TLS降级保护的有10，占比71.43%
34个DOT属于hetzner，其中受TLS降级保护的有30，占比88.24%


## 分析证书有效性 

In [133]:
print("digitalocean")
for index,value in digitalocean["certValid"].value_counts().iteritems():
    print(index,value,"---",value/len(digitalocean)*100) 
print("*******************************************")
print("apple")
for index,value in apple["certValid"].value_counts().iteritems():
    print(index,value,"---",value/len(apple)*100) 
print("*******************************************")
print("virtual")
for index,value in virtual["certValid"].value_counts().iteritems():
    print(index,value,"---",value/len(virtual)*100) 
print("*******************************************")
print("hetzner")
for index,value in hetzner["certValid"].value_counts().iteritems():
    print(index,value,"---",value/len(hetzner)*100) 

digitalocean
True 25 --- 73.52941176470588
False 9 --- 26.47058823529412
*******************************************
apple
False 4 --- 80.0
True 1 --- 20.0
*******************************************
virtual
True 7 --- 50.0
False 7 --- 50.0
*******************************************
hetzner
True 29 --- 85.29411764705883
False 5 --- 14.705882352941178


In [134]:
apple

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,caa_flag,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce
173,uklon6-doh-001.aaplimg.com,17.253.36.119,,False,read tcp 172.22.21.0:40360->17.253.36.119:443:...,,,False,none,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,uklon5-doh-002.aaplimg.com,17.253.34.247,,False,read tcp 172.22.21.0:41498->17.253.34.247:443:...,,,False,none,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,defra3-doh-001.aaplimg.com,17.253.56.119,,False,read tcp 172.22.21.0:55282->17.253.56.119:443:...,,,False,none,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,doh-test.dns.apple.com,17.253.6.247,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdQVE...,OV,True,['Apple Inc.'],True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
795,uklon6-doh-003.aaplimg.com,17.253.36.85,,False,read tcp 172.22.21.0:34354->17.253.36.85:443: ...,,,False,none,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###### 对于通过PTR记录寻找到的域名来说，可能出现域名不匹配的情况，我们不知道是该DOH的ip本身没有对应的域名还是我们没有找到


## 支持CT 

In [135]:
digitalocean_ct_num = digitalocean.query('tlsHave == True | certHave == True | ocspHave == True').ip.count()
print('{}个DOT属于digitalocean，其中支持CT的有{}，占比{:.2%}'.format(len(digitalocean),digitalocean_ct_num, digitalocean_ct_num/len(digitalocean)))

apple_ct_num = apple.query('tlsHave == True | certHave == True | ocspHave == True').ip.count()
print('{}个DOT属于apple，其中支持CT的有{}，占比{:.2%}'.format(len(apple),apple_ct_num, apple_ct_num/len(apple)))

virtual_ct_num = virtual.query('tlsHave == True | certHave == True | ocspHave == True').ip.count()
print('{}个DOT属于virtual，其中支持CT的有{}，占比{:.2%}'.format(len(virtual),virtual_ct_num, virtual_ct_num/len(virtual)))

hetzner_ct_num = hetzner.query('tlsHave == True | certHave == True | ocspHave == True').ip.count()
print('{}个DOT属于hetzner，其中支持CT的有{}，占比{:.2%}'.format(len(hetzner),hetzner_ct_num, hetzner_ct_num/len(hetzner)))

34个DOT属于digitalocean，其中支持CT的有30，占比88.24%
5个DOT属于apple，其中支持CT的有1，占比20.00%
14个DOT属于virtual，其中支持CT的有10，占比71.43%
34个DOT属于hetzner，其中支持CT的有33，占比97.06%


## 支持证书撤销 

In [136]:
digitalocean_revoke_num = digitalocean.query('crLs == True | ocsp == True | ocspStapling == True | ocspMustStaple == True').ip.count()
print('{}个DOT属于digitalocean，其中支持证书撤销的有{}，占比{:.2%}'.format(len(digitalocean),digitalocean_revoke_num, digitalocean_revoke_num/len(digitalocean)))

apple_revoke_num = apple.query('crLs == True | ocsp == True | ocspStapling == True | ocspMustStaple == True').ip.count()
print('{}个DOT属于apple，其中支持证书撤销的有{}，占比{:.2%}'.format(len(apple),apple_revoke_num, apple_revoke_num/len(apple)))

virtual_revoke_num = virtual.query('crLs == True | ocsp == True | ocspStapling == True | ocspMustStaple == True').ip.count()
print('{}个DOT属于virtual，其中支持证书撤销的有{}，占比{:.2%}'.format(len(virtual),virtual_revoke_num, virtual_revoke_num/len(virtual)))

hetzner_revoke_num = hetzner.query('crLs == True | ocsp == True | ocspStapling == True | ocspMustStaple == True').ip.count()
print('{}个DOT属于hetzner，其中支持证书撤销的有{}，占比{:.2%}'.format(len(hetzner),hetzner_revoke_num, hetzner_revoke_num/len(hetzner)))


34个DOT属于digitalocean，其中支持证书撤销的有30，占比88.24%
5个DOT属于apple，其中支持证书撤销的有1，占比20.00%
14个DOT属于virtual，其中支持证书撤销的有10，占比71.43%
34个DOT属于hetzner，其中支持证书撤销的有33，占比97.06%


## DANE-TLSA 

In [137]:
digitalocean_dane_num = digitalocean.query('tlsa_dnssec == "Secure" & dane == True').ip.count()
print('{}个DOT属于digitalocean，其中支持DANE的有{}，占比{:.2%}'.format(len(digitalocean),digitalocean_dane_num, digitalocean_dane_num/len(digitalocean)))

apple_dane_num = apple.query('tlsa_dnssec == "Secure" & dane == True').ip.count()
print('{}个DOT属于apple，其中支持DANE的有{}，占比{:.2%}'.format(len(apple),apple_dane_num, apple_dane_num/len(apple)))

virtual_dane_num = virtual.query('tlsa_dnssec == "Secure" & dane == True').ip.count()
print('{}个DOT属于virtual，其中支持DANE的有{}，占比{:.2%}'.format(len(virtual),virtual_dane_num, virtual_dane_num/len(virtual)))

hetzner_dane_num = hetzner.query('tlsa_dnssec == "Secure" & dane == True').ip.count()
print('{}个DOT属于hetzner，其中支持DANE的有{}，占比{:.2%}'.format(len(hetzner),hetzner_dane_num, hetzner_dane_num/len(hetzner)))


34个DOT属于digitalocean，其中支持DANE的有0，占比0.00%
5个DOT属于apple，其中支持DANE的有0，占比0.00%
14个DOT属于virtual，其中支持DANE的有0，占比0.00%
34个DOT属于hetzner，其中支持DANE的有0，占比0.00%


# 分析DOT的ISP

In [138]:
all_raw.isp = all_raw.isp.apply(lambda x: x.lower())

In [139]:
for index,value in all_raw["isp"].value_counts().iteritems():
    if value >= 20:
        print(index,value,value/len(all_raw)*100) 

oracle corporation 80 8.918617614269788
digitalocean, llc 44 4.9052396878483835
hetzner online gmbh 38 4.23634336677815
ovh sas 30 3.3444816053511706
none 30 3.3444816053511706
google llc 25 2.787068004459309
amazon technologies inc. 25 2.787068004459309
amazon.com, inc. 21 2.341137123745819
shenzhen tencent computer systems company limited 21 2.341137123745819
hosthatch 20 2.229654403567447


# 对国家进行分析

In [140]:
for index,value in all_raw["country"].value_counts().iteritems():
    if value >= 20:
        print(index,value,"-----",value/len(all_raw)*100) 

United States 187 ----- 20.84726867335563
Germany 129 ----- 14.381270903010032
United Kingdom 70 ----- 7.803790412486064
China 52 ----- 5.797101449275362
Japan 49 ----- 5.4626532887402455
Netherlands 46 ----- 5.128205128205128
none 30 ----- 3.3444816053511706
Canada 29 ----- 3.2329988851727984
France 25 ----- 2.787068004459309
Australia 25 ----- 2.787068004459309
Hong Kong 23 ----- 2.564102564102564
South Korea 23 ----- 2.564102564102564
Denmark 22 ----- 2.4526198439241917
Singapore 20 ----- 2.229654403567447
India 20 ----- 2.229654403567447


# 对DOT安全等级进行分析 
* **S**    &nbsp;&nbsp; 证书有效 且 TLS=1.3 且 有CT保护 且 证书未被撤销 且 有TLS降级保护 且 DANE-TLSA配置正确

* **A**    &nbsp;&nbsp; 证书有效 且 TLS=1.3 且 有CT保护 且 证书未被撤销 且 有TLS降级保护 

* **B**    &nbsp;&nbsp; 证书有效 且 TLS>=1.2

* **C**    &nbsp;&nbsp; 证书无效 或 TLS<1.2

In [27]:
def check_level(data):
    if ((data["certValid"]==True) & (data["tlsVersion"]=="TLSv1.3") & (data["flagValid"]==True) & ((data["crLs"]==True) | (data["ocsp"]==True) | (data["ocspStapling"]==True) | (data["ocspMustStaple"]==True)) & (((data["tlsVersion"]=="TLSv1.2") & (data["scsv"]==True)) | ((data["tlsVersion"]=="TLSv1.3") & (data["tls13_down_12"]==True))) & ((data["tlsa_dnssec"]=="Secure")&(data["dane"]==True))):
        return "S"
    elif ((data["certValid"]==True) & (data["tlsVersion"]=="TLSv1.3") & (data["flagValid"]==True) & ((data["crLs"]==True) | (data["ocsp"]==True) | (data["ocspStapling"]==True) | (data["ocspMustStaple"]==True)) & (((data["tlsVersion"]=="TLSv1.2") & (data["scsv"]==True)) | ((data["tlsVersion"]=="TLSv1.3") & (data["tls13_down_12"]==True)))):
        return "A"
    elif ((data["certValid"]==True) & ((data["tlsVersion"]=="TLSv1.2") | (data["tlsVersion"]=="TLSv1.3"))):
        return "B"
    elif ((data["certValid"]==False) | (data["tlsVersion"]=="TLSv1.1") | (data["tlsVersion"]=="TLSv1.0")):
        return "C"
    else:
        return "Err"

In [28]:
all_raw = all_raw.assign(grade_num=all_raw.apply(lambda x: check_level(x), axis=1))
all_raw

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce,grade_num
0,cloudflare-dns.com,1.0.0.1,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUYrVE...,OV,True,[DigiCert Inc],True,...,"{'hsts': True, 'hsts_maxage': '31536000', 'hst...",True,31536000,True,True,True,604800,"""https://report-uri.cloudflare.com/cdn-cgi/bea...",NaN,A
1,itdog.duckdns.org,168.138.209.219,TLSv1.2,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B
2,freedns.controld.dev,176.125.239.112,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdoVE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
3,dns.nextdns.io,185.253.5.128,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,[ZeroSSL],True,...,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN,A
4,dns.sb,185.37.252.149,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUdIek...,DV,True,[GoGetSSL],True,...,"{'hsts': True, 'hsts_maxage': '63072000', 'hst...",True,63072000,True,True,NaN,NaN,NaN,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,dns.seey6.xyz,60.205.210.33,,False,read tcp 172.22.21.0:57162->60.205.210.33:443:...,,,False,none,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C
721,dns.nguyendn.com,103.237.147.61,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVjVE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
722,doh.dnsfilter.com,103.247.36.150,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
723,dns.vinokurov.tk,176.107.131.185,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZKek...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A


In [29]:
all_raw.to_csv("../MAP/DOH-SP-IP-uniq.txt",index=None)

In [30]:
grade_s = all_raw[all_raw["grade_num"]=="S"]

grade_a = all_raw[all_raw["grade_num"]=="A"]

grade_b = all_raw[all_raw["grade_num"]=="B"]

grade_c = all_raw[all_raw["grade_num"]=="C"]

print('在{}个DOT中，安全等级为S的有{}，占比{:.2%}'.format(len(all_raw),len(grade_s),len(grade_s)/len(all_raw)))
print('在{}个DOT中，安全等级为A的有{}，占比{:.2%}'.format(len(all_raw),len(grade_a),len(grade_a)/len(all_raw)))
print('在{}个DOT中，安全等级为B的有{}，占比{:.2%}'.format(len(all_raw),len(grade_b),len(grade_b)/len(all_raw)))
print('在{}个DOT中，安全等级为C的有{}，占比{:.2%}'.format(len(all_raw),len(grade_c),len(grade_c)/len(all_raw)))


在725个DOT中，安全等级为S的有7，占比0.97%
在725个DOT中，安全等级为A的有419，占比57.79%
在725个DOT中，安全等级为B的有134，占比18.48%
在725个DOT中，安全等级为C的有165，占比22.76%


In [31]:
all_raw.to_csv("../doh-domain-tlscert-uniq.csv",index=None)

## 对各个等级的公司进行分析

In [32]:
grade_s

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,httpdict,hsts,hsts_maxage,hsts_include_sub,hsts_preload,expect_ct,expect_ct_maxage,expect_ct_report_uri,expect_ct_enforce,grade_num
21,anycast.censurfridns.nu,198.180.150.12,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZjak...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
109,deic-ore.anycast.uncensoreddns.org,130.226.161.34,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZhek...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
234,unicast.censurfridns.nu,89.233.43.71,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUUxRE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
259,doh.bortzmeyer.fr,193.70.85.11,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLVE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
406,anycast.censurfridns.dk,185.38.27.139,TLSv1.3,False,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZZVE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
448,dnsforge.de,176.9.93.198,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVnRE...,DV,True,[Let's Encrypt],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
543,doh.wewitro.net,46.105.230.57,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVMek...,DV,True,[ZeroSSL],True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S


In [33]:
grade_s["caa_flag"].value_counts()

False    5
True     2
Name: caa_flag, dtype: int64

In [22]:
grade_s["org"].value_counts()

Danmarks Tekniske Universitet    5
WEBPARTNER                       4
Kracon ApS                       4
Thomas Steen Rasmussen           3
RGnet OU                         2
OVH                              1
none                             1
netcup GmbH                      1
Scholz-Kallies Rene              1
Name: org, dtype: int64

In [1]:
grade_s["country"].value_counts()

NameError: name 'grade_s' is not defined

### 分析有多少S等级的DOT出现在公开数据集中 

In [24]:
pub_dot = pd.read_csv("../public/doh/have_doh_uniq.txt")
pub_dot

,ip,domain,suffix
0,1.0.0.1,1dot1dot1dot1.cloudflare-dns.com,dns-query
1,1.0.0.1,one.one.one.one,dns-query
2,1.0.0.2,security.cloudflare-dns.com,dns-query
3,1.0.0.3,family.cloudflare-dns.com,dns-query
4,101.101.101.101,dns.twnic.tw,NaN
...,...,...,...
216,9.9.9.11,dns11.quad9.net,dns-query
217,9.9.9.12,dns12.quad9.net,dns-query
218,9.9.9.13,dns13.quad9.net,dns-query
219,9.9.9.9,dns9.quad9.net,dns-query


In [25]:
pub_dot_s = grade_s[grade_s["domain"].isin(pub_dot["domain"].unique())]
pub_dot_s

,domain,ip,tlsVersion,scsv,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,tls13_down_11,tls13_down_10,http_code,http_headers,country,isp,org,tlsa_flag,caa_flag,grade_num
98646,dnsforge.de,176.9.93.198,TLSv1.3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVnRE...,DV,True,[Let's Encrypt],True,...,False,False,none,none,none,none,none,True,True,S
